In [1]:
# --- CELL 1 AVU_ignition_1.ipynb: Global Setup & Parameter Intake (Papermill/Env/UI-safe) ---
import os
import json
import warnings
from datetime import datetime, date, timedelta
from pathlib import Path
from typing import Any, Dict
import numpy as np
import pandas as pd

def _to_py(o):
    if o is None: return None
    if isinstance(o, (np.integer,)):  return int(o)
    if isinstance(o, (np.floating,)): return None if np.isnan(o) else float(o)
    if isinstance(o, (np.bool_,)):    return bool(o)
    if isinstance(o, (pd.Timestamp,)):return o.isoformat()
    return o

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# -------------------------------
# 0) Constants used across the run
# -------------------------------
NUM_SLOTS = 5
DAYS_FULL = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
DAYS_LOWER = [d.lower() for d in DAYS_FULL]

# When preferences can't be fully satisfied, later cells can consult these knobs
ENGINE_FILL_POLICY = {
    "min_slots_per_day": NUM_SLOTS,   # aim to fill all visible slots
    "relaxation_enabled": True,
    "relaxation_order": [
        # later cells should progressively relax in this order until enough wines found:
        "loyalty",          # treat 'all' if too strict
        "wine_type",        # allow any type if filter blocks results
        "bottle_size",      # allow any size if specific format too rare
        "price_tier",       # widen tier ranges (Budget→Mid→Premium→Luxury→Ultra)
        "last_stock_only",  # ignore last-stock constraint if needed
        "seasonality"       # ignore seasonality boost as hard constraint (keep as soft weight)
    ],
}

# -------------------------------
# 1) Try to read Papermill parameters (if any)
# -------------------------------
_pm_params: Dict[str, Any] = {}
try:
    from papermill import get_parameters  # type: ignore
    _pm_params = get_parameters() or {}
except Exception:
    _pm_params = {}

def _coerce_int(value, fallback):
    try:
        return int(value)
    except Exception:
        return int(fallback)

def _iso_week_now():
    try:
        return datetime.now().isocalendar().week
    except Exception:
        return int(datetime.now().strftime("%V"))

# -------------------------------
# 2) Resolve week_number (priority: papermill -> env -> runtime default)
# -------------------------------
if "week_number" in globals():
    _week_candidate = globals().get("week_number")
elif "week_number" in _pm_params:
    _week_candidate = _pm_params.get("week_number")
else:
    _week_candidate = os.getenv("WEEK_NUMBER", _iso_week_now())

week_number = _coerce_int(_week_candidate, _iso_week_now())
# Clamp to ISO range
if week_number < 1 or week_number > 53:
    week_number = _iso_week_now()

# Useful week helpers
def week_bounds(iso_week: int, year: int = datetime.now().year):
    # Monday as first day of ISO week
    d = date.fromisocalendar(year, iso_week, 1)
    return d, d + timedelta(days=6)

week_start_date, week_end_date = week_bounds(week_number)

# --- YEAR AWARENESS + EUROPE/ZURICH TZ + PARAM SUPPORT ---
from zoneinfo import ZoneInfo

def _iso_week_now_europe():
    now = datetime.now(ZoneInfo("Europe/Zurich"))
    iso = now.isocalendar()
    return iso.year, iso.week

# Allow papermill/env to pass an explicit calendar year
if "calendar_year" in globals():
    _year_candidate = globals().get("calendar_year")
elif "calendar_year" in _pm_params:
    _year_candidate = _pm_params.get("calendar_year")
else:
    _year_candidate = os.getenv("CALENDAR_YEAR", None)

try:
    calendar_year = int(_year_candidate) if _year_candidate is not None else _iso_week_now_europe()[0]
except Exception:
    calendar_year = _iso_week_now_europe()[0]

# Week bounds should use the chosen calendar_year
def week_bounds(iso_week: int, year: int):
    # Guard invalid 53rd week for some years
    try:
        d = date.fromisocalendar(year, iso_week, 1)
    except ValueError:
        iso_week = min(max(iso_week, 1), 52)
        d = date.fromisocalendar(year, iso_week, 1)
    return d, d + timedelta(days=6)

week_start_date, week_end_date = week_bounds(week_number, calendar_year)

# -------------------------------
# 3) Resolve IO paths (priority: papermill -> env -> sensible defaults)
# -------------------------------
def _resolve_path(param_key, env_key, default_path_str):
    if param_key in globals():
        p = globals().get(param_key)
    elif param_key in _pm_params:
        p = _pm_params.get(param_key)
    else:
        p = os.getenv(env_key, default_path_str)
    return Path(p)

# Defaults match your project structure
_default_source = str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "SOURCE_FILES")
_default_output = str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA")

SOURCE_PATH: Path = _resolve_path("input_path", "INPUT_PATH", _default_source)
OUTPUT_PATH: Path = _resolve_path("output_path", "OUTPUT_PATH", _default_output)

# Make sure all dirs exist (incl. subfolders we’ll use)
SOURCE_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
LOCKED_PATH: Path = OUTPUT_PATH / "locked_weeks"
CALENDAR_PATH: Path = OUTPUT_PATH / "calendar"       # where schedule JSONs per week can live
EXPORTS_PATH: Path = OUTPUT_PATH / "exports"         # generic exports
POWERBI_PATH: Path = OUTPUT_PATH / "powerbi"         # enriched Excel for Power BI
TMP_PATH: Path = OUTPUT_PATH / "_tmp"

for p in (LOCKED_PATH, CALENDAR_PATH, EXPORTS_PATH, POWERBI_PATH, TMP_PATH):
    p.mkdir(parents=True, exist_ok=True)

# Optional reports path (kept for compatibility)
REPORTS_PATH: Path = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "non_recipient_reports"

# -------------------------------
# 4) Load lightweight UI inputs if present (non-fatal if missing)
#    These files are written by the Flask app before launching the notebook.
# -------------------------------
NOTEBOOKS_DIR = Path("notebooks")
FILTERS_FILE = NOTEBOOKS_DIR / "filters.json"
LOCKED_SNAPSHOT_FILE = NOTEBOOKS_DIR / "locked_calendar.json"  # snapshot of UI locks passed in

def _load_json_or_empty(path: Path):
    try:
        if path.exists() and path.stat().st_size > 0:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
    except Exception:
        pass
    return {}

raw_filters: Dict[str, Any] = _load_json_or_empty(FILTERS_FILE)              # may be {}
_locked_raw = _load_json_or_empty(LOCKED_SNAPSHOT_FILE)                       # may be {}

# -------- Canonicalize UI/legacy locks to one shape --------
# Accepts:
#   - {"weekly_calendar": {day: [slot...]}}
#   - {day: [slot...]}
#   - {day: {"main":[slot...], "overflow":[...]}}
# Returns:
#   {day: {"main": [None-padded to NUM_SLOTS], "overflow": list}}
def _coerce_locked_snapshot(snap, days=None, num_slots=NUM_SLOTS):
    if days is None:
        days = DAYS_FULL

    # unwrap {"weekly_calendar": {...}} if present
    if isinstance(snap, dict) and isinstance(snap.get("weekly_calendar"), dict):
        snap = snap["weekly_calendar"]

    out = {}
    for d in days:
        node = (snap or {}).get(d) or (snap or {}).get(d.lower())
        main, overflow = [], []
        if isinstance(node, list):
            main = node
        elif isinstance(node, dict):
            if isinstance(node.get("main"), list):
                main = node.get("main") or []
            elif isinstance(node.get("slots"), list):
                main = node.get("slots") or []
            else:
                # tolerate indexed dicts {0:...,1:...}
                tmp = []
                for i in range(num_slots):
                    v = node.get(i, node.get(str(i)))
                    tmp.append(v if v else None)
                main = tmp
            if isinstance(node.get("overflow"), list):
                overflow = node.get("overflow") or []
        # clamp & pad main
        main = (main or [])[:num_slots]
        if len(main) < num_slots:
            main = main + [None] * (num_slots - len(main))
        out[d] = {"main": [x if x else None for x in main], "overflow": list(overflow or [])}
    return out

def _load_json(path: Path):
    try:
        if path.exists() and path.stat().st_size > 0:
            return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

# Normalize the incoming UI snapshot
LOCKED_SNAPSHOT_NORM = _coerce_locked_snapshot(_locked_raw, DAYS_FULL, NUM_SLOTS)

# If empty, fallback to persisted week file
def _is_empty_main(snap_norm):
    return not any(any(snap_norm[d]["main"]) for d in DAYS_FULL)

if _is_empty_main(LOCKED_SNAPSHOT_NORM):
    persisted = _load_json(LOCKED_PATH / f"locked_calendar_{calendar_year}_week_{week_number}.json")
    LOCKED_SNAPSHOT_NORM = _coerce_locked_snapshot(persisted, DAYS_FULL, NUM_SLOTS)

# Export *both* forms for downstream cells:
# - EFFECTIVE_LOCKS / LOCKED_CALENDAR: {day: [slot,..]} (Cell 6/7 expect this)
# - locked_calendar_snapshot: {day: {"main":[...], "overflow":[...]}} (for any structured readers)
EFFECTIVE_LOCKS = {d: LOCKED_SNAPSHOT_NORM[d]["main"] for d in DAYS_FULL}
LOCKED_CALENDAR = dict(EFFECTIVE_LOCKS)  # alias used by some cells
locked_calendar_snapshot = dict(LOCKED_SNAPSHOT_NORM)  # keep structured version for completeness

# -------------------------------
# 5) Helpers: normalize filters & day keys
# -------------------------------
PRICE_BUCKETS = ["Budget","Mid-range","Premium","Luxury","Ultra Luxury"]

def price_tier_label(price: float) -> str:
    try:
        p = float(price)
    except Exception:
        return ""
    if p < 50: return "Budget"
    elif p < 100: return "Mid-range"
    elif p < 200: return "Premium"
    elif p < 500: return "Luxury"
    else: return "Ultra Luxury"

DEFAULT_FILTERS: Dict[str, Any] = {
    "loyalty": "all",
    "wine_type": None,        # None = all
    "bottle_size": 750,       # ml
    "price_tier_bucket": "",  # "" = all
    "last_stock": False,
    "last_stock_threshold": 10,
    "seasonality_boost": False,
    "style": "default",
    "calendar_day": None
}

def _as_bool(x): 
    if isinstance(x, bool): return x
    if isinstance(x, (int, float)): return bool(x)
    if isinstance(x, str): return x.strip().lower() in {"1","true","yes","y","on"}
    return False

def _as_int(x, default=None):
    try:
        return int(x)
    except Exception:
        return default

def normalize_filters(f: Dict[str, Any]) -> Dict[str, Any]:
    f = dict(f or {})
    out = dict(DEFAULT_FILTERS)

    out["loyalty"] = str(f.get("loyalty", out["loyalty"])).strip().lower() or "all"

    wt = f.get("wine_type", out["wine_type"])
    if isinstance(wt, str) and wt.strip().lower() == "all":
        wt = None
    out["wine_type"] = wt

    out["bottle_size"] = _as_int(f.get("bottle_size", out["bottle_size"]), 750)

    pt = str(f.get("price_tier_bucket", out["price_tier_bucket"])).strip()
    out["price_tier_bucket"] = pt if pt in PRICE_BUCKETS or pt == "" else ""

    out["last_stock"] = _as_bool(f.get("last_stock", out["last_stock"]))
    out["last_stock_threshold"] = _as_int(f.get("last_stock_threshold", out["last_stock_threshold"]), 10)

    out["seasonality_boost"] = _as_bool(f.get("seasonality_boost", out["seasonality_boost"]))

    style = str(f.get("style", out["style"])).strip().lower()
    out["style"] = style if style in {"default","cat","nigo"} else "default"

    cd = f.get("calendar_day", None)
    out["calendar_day"] = str(cd) if cd else None

    return out

filters = normalize_filters(raw_filters)

# -------------------------------
# 6) Echo configuration
# -------------------------------
print(f"✅ Source data path:   {SOURCE_PATH.resolve()}")
print(f"✅ Output data path:   {OUTPUT_PATH.resolve()}")
print(f"📦 Locked weeks path:  {LOCKED_PATH.resolve()}")
print(f"📄 Exports path:       {EXPORTS_PATH.resolve()}")
print(f"📊 PowerBI path:       {POWERBI_PATH.resolve()}")
print(f"📅 Using ISO Week:     {week_number} ({week_start_date} → {week_end_date})")
print(f"🧩 Filters provided:   {'yes' if raw_filters else 'no'}   → resolved: {filters}")
print(f"🔒 Locked snapshot:    {'yes' if any(EFFECTIVE_LOCKS.get(d) for d in DAYS_FULL) else 'no'}")
print("✅ Environment & parameters initialized.")


✅ Source data path:   C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\SOURCE_FILES
✅ Output data path:   C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA
📦 Locked weeks path:  C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\locked_weeks
📄 Exports path:       C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\exports
📊 PowerBI path:       C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\powerbi
📅 Using ISO Week:     38 (2025-09-15 → 2025-09-21)
🧩 Filters provided:   no   → resolved: {'loyalty': 'all', 'wine_type': None, 'bottle_size': 750, 'price_tier_bucket': '', 'last_stock': False, 'last_stock_threshold': 10, 'seasonality_boost': False, 'style': 'default', 'calendar_day': None}
🔒 Locked snapshot:    yes
✅ Environment & parameters initialized.


In [2]:
# --- CELL 2: Set Occasion/Subfolder Path ---
# Creates a deterministic subfolder for this run inside REPORTS_PATH.
# Safe with/without papermill, and robust if OCCASION is empty.

from datetime import datetime, timedelta, date
from pathlib import Path
import os
import re

# Fallback for _iso_week_now_europe if Cell 1 wasn't executed
try:
    _iso_week_now_europe  # type: ignore
except NameError:
    try:
        from zoneinfo import ZoneInfo
    except Exception:
        ZoneInfo = None  # type: ignore
    def _iso_week_now_europe():
        now = datetime.now(ZoneInfo("Europe/Zurich")) if ZoneInfo else datetime.now()
        iso = now.isocalendar()
        return iso.year, iso.week

# Use selected week/year from Cell 1 when available
try:
    _wk = int(week_number)
except Exception:
    _wk = _iso_week_now_europe()[1]

try:
    _year = int(calendar_year)
except Exception:
    _year = _iso_week_now_europe()[0]

def _week_bounds(iso_week: int, year: int):
    try:
        start = date.fromisocalendar(year, iso_week, 1)
    except ValueError:
        start = date.fromisocalendar(year, min(max(iso_week, 1), 52), 1)
    end = start + timedelta(days=6)
    return start, end

# Prefer globals from Cell 1 if present
try:
    _week_start = week_start_date; _week_end = week_end_date
except Exception:
    _week_start, _week_end = _week_bounds(_wk, _year)

def _season_from_week(week: int) -> str:
    if 9 <= week <= 21:
        return "Spring"
    elif 22 <= week <= 35:
        return "Summer"
    elif 36 <= week <= 48:
        return "Autumn"
    else:
        return "Winter"

_season = _season_from_week(_wk)

# --- Read OCCASION from globals/papermill/env; keep your default ---
try:
    OCCASION  # type: ignore  # if already defined above, leave it
except NameError:
    OCCASION = os.getenv("OCCASION", "Notreceived")

# Optional “forced_day” kept for compatibility; ignored if falsy
try:
    forced_day  # type: ignore
except NameError:
    forced_day = None

def _slugify(s: str) -> str:
    s = (s or "").strip()
    if not s:
        return ""
    s = s.replace("&", "and").replace("/", "-")
    s = re.sub(r"[^A-Za-z0-9_-]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or ""

# Compose a stable, human-readable folder name
_occ_slug = _slugify(str(OCCASION))
_base = f"{_year}_W{_wk:02d}_{_season}"
folder_name = f"{_base}_{_occ_slug}" if _occ_slug else (_slugify(str(forced_day)) or _base)

# Ensure REPORTS_PATH exists (from Cell 1); fall back to IRON_DATA/non_recipient_reports if missing
try:
    REPORTS_PATH
except NameError:
    REPORTS_PATH = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "non_recipient_reports"
REPORTS_PATH.mkdir(parents=True, exist_ok=True)

report_subfolder = REPORTS_PATH / folder_name
report_subfolder.mkdir(parents=True, exist_ok=True)

# Also expose a canonical var some downstream cells may prefer
REPORT_RUN_PATH = report_subfolder

print(
    f"📅 Week {_wk} "
    f"({_week_start.strftime('%b %d')} – {_week_end.strftime('%b %d')}) "
    f"| Season: {_season}"
)
print(f"🗂  Occasion: {OCCASION!r}  → Folder: {folder_name}")
print(f"✅ Report subfolder ready at: {report_subfolder}")


📅 Week 38 (Sep 15 – Sep 21) | Season: Autumn
🗂  Occasion: 'Notreceived'  → Folder: 2025_W38_Autumn_Notreceived
✅ Report subfolder ready at: C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\non_recipient_reports\2025_W38_Autumn_Notreceived


In [3]:
# --- CELL 3: Load & Filter Clients for Targeting (Cloud-Safe) ---
# Loads Lines.xlsx and Power BI stats, cleans & filters, and writes a compact
# pickle for downstream cells. Robust to Excel float IDs (e.g., "12345.0").

from pathlib import Path
import pandas as pd
import numpy as np

# ---------- Inputs / Outputs ----------
lines_path = SOURCE_PATH / "Lines.xlsx"
stats_path = SOURCE_PATH / "Power BI Dtld. Statistics ALL.xlsx"
output_temp_path = OUTPUT_PATH / "filtered_clients.pkl"

def _ensure_file(p: Path, label: str):
    if not p.exists() or p.stat().st_size == 0:
        print(f"❌ Missing or empty {label}: {p}")
        return False
    return True

def _norm_id_series(s: pd.Series) -> pd.Series:
    """Normalize Excel-ish ID columns:
       - cast to string, strip
       - drop trailing '.0'
       - collapse common null tokens to NaN
    """
    if s is None:
        return pd.Series(dtype="string")
    out = (
        s.astype(str).str.strip()
         .str.replace(r"\.0$", "", regex=True)
         .replace({r"^(nan|na|null|none|\s*)$", np.nan}, regex=True, inplace=False)
    )
    return out

# ---------- Load ----------
ok_lines = _ensure_file(lines_path, "Lines.xlsx")
ok_stats = _ensure_file(stats_path, "Power BI Stats.xlsx")

if not (ok_lines and ok_stats):
    # Write empty output so downstream cells can still run gracefully
    empty = pd.DataFrame(columns=["customer_no","spanish_customer_no","total_bottle_amount_lcy"])
    empty.to_pickle(output_temp_path)
    clients_df = empty.copy()
    print("⚠️ One or more inputs missing. Wrote empty filtered_clients.pkl and continuing.")
else:
    # Read Excel (engine chosen by pandas)
    lines_df = pd.read_excel(lines_path)
    stats_df = pd.read_excel(stats_path)

    # Clean column names
    lines_df.columns = lines_df.columns.str.strip()
    stats_df.columns = stats_df.columns.str.strip()

    print("📋 Columns in Lines:", lines_df.columns.tolist())
    print("📋 Columns in Stats:", stats_df.columns.tolist())

    # ---------- Validate & Filter (Excluded) ----------
    if "Excluded" not in lines_df.columns:
        raise KeyError("❌ 'Excluded' column not found in Lines.xlsx.")
    
    excluded_norm = lines_df["Excluded"].apply(lambda v: str(v).strip().lower() if pd.notna(v) else "")
    # Treat only explicit truthy values as excluded
    truthy_exclude = {"true", "1", "yes", "y", "t"}
    keep_mask = ~excluded_norm.isin(truthy_exclude)
    filtered_lines = lines_df.loc[keep_mask].copy()
    print(f"🧮 Filtered lines count: {len(filtered_lines)}")
    if filtered_lines.empty:
        print("⚠️ No lines passed the Excluded=False filter.")

    # ---------- Extract and normalize contact numbers ----------
    contact_col = None
    for c in ["Contact No.", "Contact No", "Customer No.", "Customer No", "ContactNo", "CustomerNumber"]:
        if c in filtered_lines.columns:
            contact_col = c
            break
    if contact_col is None:
        raise KeyError("❌ 'Contact No.' column not found in Lines.xlsx.")

    contact_nos = _norm_id_series(filtered_lines[contact_col]).dropna().unique().tolist()
    print("🔍 Sample Contact Nos:", contact_nos[:5])

    # ---------- Validate required stats columns ----------
    required_cols = ["Customer No.", "Spanish Customer No.", "Total Bottle Amount (LCY)"]
    missing = [c for c in required_cols if c not in stats_df.columns]
    if missing:
        raise KeyError(f"❌ Missing columns in Power BI Stats file: {missing}")

    # Normalize IDs and amounts
    stats_df["Customer No."] = _norm_id_series(stats_df["Customer No."])
    stats_df["Spanish Customer No."] = _norm_id_series(stats_df["Spanish Customer No."])
    stats_df["Total Bottle Amount (LCY)"] = pd.to_numeric(
        stats_df["Total Bottle Amount (LCY)"], errors="coerce"
    ).fillna(0.0)

    # ---------- Business rule for client 122636 (vectorized) ----------
    # If Customer No. == '122636' AND Spanish Customer No. is one of contact_nos:
    #   - overwrite Customer No. with Spanish Customer No.
    #   - multiply Total Bottle Amount (LCY) by 1.12
    mask_122636 = (stats_df["Customer No."] == "122636") & (
        stats_df["Spanish Customer No."].isin(contact_nos)
    )
    if mask_122636.any():
        stats_df.loc[mask_122636, "Customer No."] = stats_df.loc[mask_122636, "Spanish Customer No."]
        stats_df.loc[mask_122636, "Total Bottle Amount (LCY)"] *= 1.12
        print(f"🔧 Applied 1.12x adjustment to {mask_122636.sum()} rows for client 122636.")

    # ---------- Final filter: keep only valid clients ----------
    final_df = stats_df.loc[
        stats_df["Customer No."].isin(contact_nos)
    ].copy()

    # ---------- Normalize output schema ----------
    final_df = final_df.rename(
        columns={
            "Customer No.": "customer_no",
            "Spanish Customer No.": "spanish_customer_no",
            "Total Bottle Amount (LCY)": "total_bottle_amount_lcy",
        }
    )
    # keep only the essentials + anything you know downstream needs
    core_cols = ["customer_no", "spanish_customer_no", "total_bottle_amount_lcy"]
    # If there are extra useful columns, keep them but put core first
    extras = [c for c in final_df.columns if c not in core_cols]
    final_df = final_df[core_cols + extras]

    # Persist compact binary for downstream cells
    final_df.to_pickle(output_temp_path)

    # Expose for the rest of the notebook
    clients_df = final_df

    # ---------- Logs ----------
    print(f"✅ Final shape: {final_df.shape}")
    print(f"📁 Internal .pkl saved to: {output_temp_path}")
    if final_df.empty:
        print("⚠️ The output file contains only headers — no matching customers found.")
    else:
        print("👀 Sample rows:")
        try:
            from IPython.display import display  # safe if IPython present
            display(final_df.head(3))
        except Exception:
            print(final_df.head(3).to_string(index=False))


📋 Columns in Lines: ['Salesperson Code', 'Total Order No', 'Last Offer Sent Date', 'Turnover (€)', 'Sent Emails Today', 'Sent Emails Scheduled', 'Excluded', 'Unsubscribed', 'Contact Company Name', 'Contact Person Name', 'Country', 'City', 'Language Code', 'Contact Email', 'Contact Email CC', 'Email Sent', 'Send Email Error', 'Contact Phone No.', 'Contact Mobile Phone No.', 'Informal', 'Contact No.', 'Included as New Customer', 'First Purchase Date', 'Bought Wine with exceed price', 'Bought Wine in Parent Campaign(Recall)', 'Unsubsc. for Recall', 'Buy from Competitor', 'Cont. Offer Currency']
📋 Columns in Stats: ['Period', 'Country Code', 'Customer Post Code', 'Customer City', 'Customer Net Worth', 'Spanish Customer No.', 'Spanish Customer Name', 'Wine Country', 'Wine Overall Type', 'Wine Region', 'Wine Sub-Region', 'Wine Type', 'Classification', 'Color Code', 'Producer No.', 'Wine No.', 'Wine Name', 'Producer Name', 'Item Sales Price from Price List', 'Item No.', 'Customer No.', 'Custo

,customer_no,spanish_customer_no,total_bottle_amount_lcy,Period,Country Code,Customer Post Code,Customer City,Customer Net Worth,Spanish Customer Name,Wine Country,...,Item Real Stock,Item Stock,Campaign No.,Document Currency Factor,Document Canceled,PA,PV Total,Sales Source,Sales Sub-Source,Salesperson
1570,10044,nan,411.655874,2025,LI,9490,Vaduz,0,NaN,IT,...,0,0,CM-25-02316,0,False,0,0,NaN,NaN,C.NETTER
1586,101332,nan,1108.912355,2025,LI,9494,Schaan,0,NaN,IT,...,0,0,CM-25-01069,0,False,0,0,NaN,NaN,N.BOLDRINI
1629,101332,nan,1463.591049,2025,LI,9494,Schaan,0,NaN,US,...,0,0,CM-25-00459,0,False,0,0,NaN,NaN,N.BOLDRINI


In [4]:
# --- CELL 4: Enhance Stock Dataset (Detailed Stock + OMT Info) ---
# Robust load, normalize, enrich with price tiers (label + stable key),
# trait inference, and OMT campaign summary. Outputs stock_df_final.pkl.

from pathlib import Path
import pandas as pd, numpy as np, re, json


# Part 1 - Turn OFF legacy history writer; Cell 5 will own history + schedules
ENABLE_LEGACY_HISTORY_WRITER = False

# --- Bootstrap IO paths if Cell 1 wasn't run ---
try:
    OUTPUT_PATH
    SOURCE_PATH
except NameError:
    from pathlib import Path
    import os
    def _resolve_path(env_key, default_path_str):
        return Path(os.getenv(env_key, default_path_str))
    SOURCE_PATH = _resolve_path("INPUT_PATH", str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "SOURCE_FILES"))
    OUTPUT_PATH = _resolve_path("OUTPUT_PATH", str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"))
    SOURCE_PATH.mkdir(parents=True, exist_ok=True)
    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# (…rest of your Cell 4 code unchanged…)

if ENABLE_LEGACY_HISTORY_WRITER:
    pass  # intentionally disabled (history is built in Cell 5)

# --- Safe unidecode import (fallback) ---
try:
    from unidecode import unidecode  # pip install Unidecode
except Exception:
    import unicodedata
    def unidecode(x):
        x = '' if x is None else str(x)
        return ''.join(
            ch for ch in unicodedata.normalize('NFKD', x)
            if not unicodedata.combining(ch)
        )

stock_path = SOURCE_PATH / "Detailed Stock List.xlsx"
omt_path   = SOURCE_PATH / "OMT Main Offer List.xlsx"

def _ensure_file(p: Path, label: str) -> bool:
    if not p.exists() or p.stat().st_size == 0:
        print(f"❌ Missing or empty {label}: {p}")
        return False
    return True

ok_stock = _ensure_file(stock_path, "Detailed Stock List.xlsx")
ok_omt   = _ensure_file(omt_path, "OMT Main Offer List.xlsx")

if not ok_stock:
    # Minimal empty frame so downstream cells can still run.
    stock_df = pd.DataFrame(columns=[
        "id","wine","producer","classification","region","type","color","size","vintage",
        "origin","stock","CHF Price","price_tier","price_tier_key","full_type","region_group",
        "bottle_size_ml","grape_list","body","sweetness","occasion","wine_full_match_string",
        "Num_of_CM","most_recent_date","last_eur_price","last_chf_price","number_of_sent_emails"
    ])
    final_stock_path = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df.to_pickle(final_stock_path)
    print(f"⚠️ No stock file. Wrote empty dataset to: {final_stock_path}")
else:
    # ---- Load Detailed Stock (header fallback: row 2 first, then row 0) ----
    try:
        detailed_df = pd.read_excel(stock_path, header=2)
        # If the expected key columns aren't present, retry with header=0
        if not any(k in detailed_df.columns for k in ["ID","Item No.","Wine","Stock"]):
            detailed_df = pd.read_excel(stock_path, header=0)
    except Exception:
        detailed_df = pd.read_excel(stock_path, header=0)

    # ---- Flexible rename map (accept common variants) ----
    print("📋 Stock columns (raw):", detailed_df.columns.tolist())
    rename_map_variants = {
        "ID": "id",
        "Item No.": "id",
        "Wine": "wine",
        "Producer": "producer",
        "Classification": "classification",
        "Region": "region",
        "Type": "type",
        "Color": "color",
        "Size": "size",
        "Vintage": "vintage",
        "Origin": "origin",
        "Stock": "stock",
        "Qty": "stock",
        "CHF p/bt VAT excl.": "CHF Price",
        "CHF p/bt (VAT excl.)": "CHF Price",
        "Unit Price (CHF)": "CHF Price",
        "CHF Price": "CHF Price",
        "AVG": "avg_score",
        "Average Score": "avg_score",
    }
    actual_map = {c: rename_map_variants[c] for c in detailed_df.columns if c in rename_map_variants}
    detailed_df = detailed_df.rename(columns=actual_map)

    # ---- Ensure core columns exist ----
    for c in ["id","wine","producer","classification","region","type","color","size","vintage","origin","stock","CHF Price","avg_score"]:
        if c not in detailed_df.columns:
            detailed_df[c] = np.nan

    # ---- Normalize critical fields ----
    detailed_df["id"] = (
        detailed_df["id"].astype(str).str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .replace({"nan": "", "None": ""})
    )
    # CHF Price → numeric
    detailed_df["CHF Price"] = (
        detailed_df["CHF Price"]
        .astype(str).str.replace(r"[^\d\.,-]", "", regex=True)
        .str.replace(",", ".", regex=False)
    )
    detailed_df["CHF Price"] = pd.to_numeric(detailed_df["CHF Price"], errors="coerce")
    # Stock → numeric, missing → 0
    detailed_df["stock"] = pd.to_numeric(detailed_df["stock"], errors="coerce").fillna(0)

    # ---- OMT Campaign Summary (robust headers & normalization) ----
    if not ok_omt:
        campaign_summary = pd.DataFrame(columns=[
            "Item No.","Num_of_CM","most_recent_date","last_eur_price","last_chf_price","number_of_sent_emails"
        ])
    else:
        omt_df = pd.read_excel(omt_path)

        # Canonical headers
        id_aliases    = ["Item No.","Item No","Item","ID","Wine ID","Sku","SKU","SKU Code","Item Code","Product ID","Code"]
        sched_aliases = ["Schedule DateTime","Scheduled","Scheduled At","DateTime","Date","Send Date","Offer Date",
                         "Campaign Date","Scheduled Date","Scheduled On","Sent At","Created","Execution Date"]
        eur_price_aliases = ["Unit Price (EUR)","EUR Unit Price","EUR Price","Price (EUR)"]
        chf_price_aliases = ["Unit Price (CHF)","CHF Unit Price","CHF Price","Unit Price"]
        sent_aliases  = ["Number of Sent Emails","Sent Emails","Emails Sent"]

        def _pick(cands, df):
            for c in cands:
                if c in df.columns: return c
            return None

        id_col    = _pick(id_aliases, omt_df)
        if id_col is None:
            campaign_summary = pd.DataFrame(columns=[
                "Item No.","Num_of_CM","most_recent_date","last_eur_price","last_chf_price","number_of_sent_emails"
            ])
        else:
            sched_col = _pick(sched_aliases, omt_df)
            eur_col   = _pick(eur_price_aliases, omt_df)
            chf_col   = _pick(chf_price_aliases, omt_df)
            sent_col  = _pick(sent_aliases, omt_df)

            omt_df = omt_df.rename(columns={id_col: "Item No."})
            omt_df["Item No."] = (
                omt_df["Item No."].astype(str).str.strip()
                .str.replace(r"\.0$", "", regex=True)
            )

            if sched_col:
                omt_df["Schedule DateTime"] = pd.to_datetime(omt_df[sched_col], errors="coerce", dayfirst=True, utc=False)
            else:
                omt_df["Schedule DateTime"] = pd.NaT

            omt_df["Unit Price (EUR)"] = pd.to_numeric(omt_df[eur_col], errors="coerce") if eur_col else np.nan
            omt_df["Unit Price"]       = pd.to_numeric(omt_df[chf_col], errors="coerce") if chf_col else np.nan
            omt_df["Number of Sent Emails"] = pd.to_numeric(omt_df[sent_col], errors="coerce") if sent_col else np.nan

            # ---- CLUSTERING across size variants on the same timestamp ----
            desc_col = next((c for c in ["Item Description","Description","Product Name","Wine","Title"] if c in omt_df.columns), None)
            def _norm_desc(s: str) -> str:
                s = str(s or "").strip()
                # strip trailing “, 75cl” / “, 1.5L” / “, 750” / bare digits
                s = re.sub(r"(,\s*)?(\d+(\.\d+)?\s*(ml|cl|l|lt))\s*$", "", s, flags=re.I)
                s = re.sub(r"(,\s*)?\d{2,4}\s*$", "", s)
                return re.sub(r"\s+", " ", s).strip()

            omt_df["__desc_base"] = omt_df[desc_col].map(_norm_desc) if desc_col else ""
            vint_col = next((c for c in ["Vintage","Year"] if c in omt_df.columns), None)
            if vint_col:
                omt_df[vint_col] = omt_df[vint_col].astype(str).str.strip().replace({"": "NV", "nan": "NV"})
            omt_df["__ts_min"] = pd.to_datetime(omt_df["Schedule DateTime"], errors="coerce").dt.floor("min")
            omt_df["__name_key"] = np.where(
                omt_df["__desc_base"].ne(""),
                omt_df["__desc_base"] + "::" + (omt_df[vint_col] if vint_col else "NV"),
                ""
            )

            # Cluster by (name_key, timestamp) -> sum sent emails, keep last prices
            name_clusters = (
                omt_df[omt_df["__name_key"].ne("")]
                .groupby(["__name_key","__ts_min"], dropna=False)
                .agg(number_of_sent_emails=("Number of Sent Emails","sum"),
                     last_eur_price=("Unit Price (EUR)","last"),
                     last_chf_price=("Unit Price","last"))
                .reset_index()
            )
            # Latest per name_key
            name_latest = (
                name_clusters.sort_values("__ts_min")
                .groupby("__name_key", as_index=False).tail(1)
                .rename(columns={"__ts_min":"most_recent_date"})
            )

            # ---- Classic per-ID summary (kept; we still merge by id) ----
            omt_df = omt_df.sort_values(["Item No.","Schedule DateTime"], ascending=[True, True])
            counts = omt_df.groupby("Item No.").size().rename("Num_of_CM")
            last_rows = omt_df.groupby("Item No.").tail(1)[
                ["Item No.","Schedule DateTime","Unit Price (EUR)","Unit Price","Number of Sent Emails"]
            ]

            campaign_summary = (
                last_rows.rename(columns={
                    "Schedule DateTime": "most_recent_date",
                    "Unit Price (EUR)": "last_eur_price",
                    "Unit Price": "last_chf_price",
                    "Number of Sent Emails": "number_of_sent_emails",
                })
                .merge(counts, left_on="Item No.", right_index=True, how="left")
            )

            # Expose for later index build (Cell 5 may use this if present)
            _OMT_NAME_LATEST = name_latest.copy()

    # ---- Price tier logic (label + stable key) ----
    def price_tier_label(price):
        try:
            p = float(price)
            if p < 50:   return "Budget"
            if p < 100:  return "Mid-range"
            if p < 200:  return "Premium"
            if p < 500:  return "Luxury"
            return "Ultra Luxury"
        except Exception:
            return None

    def price_tier_key_from_label(label):
        m = (label or "").strip().lower()
        return {
            "budget": "budget",
            "mid-range": "mid",
            "premium": "premium",
            "luxury": "luxury",
            "ultra luxury": "ultra",
        }.get(m, ""

)

# Part 2 -     
    
    detailed_df["price_tier"] = detailed_df["CHF Price"].apply(price_tier_label)
    detailed_df["price_tier_key"] = detailed_df["price_tier"].apply(price_tier_key_from_label)

    # ---- Merge OMT summary → stock ----
    enhanced_df = detailed_df.merge(
        campaign_summary, how="left", left_on="id", right_on="Item No."
    )

    # Region group fallback (handle empty strings)
    enhanced_df["region_group"] = (
        enhanced_df["origin"].replace("", np.nan)
        .fillna(enhanced_df["region"])
    )
    enhanced_df["most_recent_date"] = pd.to_datetime(enhanced_df["most_recent_date"], errors="coerce")

    # ---- Helpers / inference ----
    def infer_type_class(row):
        t = unidecode(str(row.get("type", "")).lower())
        c = unidecode(str(row.get("color", "")).lower())
        txt = f"{t} {c}"
        if "sparkling" in txt or "champagne" in txt or "cava" in txt or "prosecco" in txt:
            return "Sparkling"
        if "dessert" in txt or "sweet" in txt or "sauternes" in txt or "porto" in txt:
            return "Dessert"
        if "rose" in txt or "rosé" in txt:
            return "Rose"
        if "white" in txt or "blanc" in txt or "bianco" in txt:
            return "White"
        if "red" in txt or "rouge" in txt or "rosso" in txt:
            return "Red"
        return "Red" if "red" in t else ("White" if "white" in t else "Red")

    def parse_bottle_size_ml(val):
        s = unidecode(str(val)).lower().strip()
        if not s or s == "nan": return np.nan
        if "jeroboam" in s: return 3000
        if "magnum" in s:   return 1500
        s = s.replace("lt", "l")
        if "l" in s and "ml" not in s and "cl" not in s:
            num = re.sub(r"[^\d\.]", "", s)
            try: return int(float(num) * 1000)
            except Exception: return np.nan
        s = s.replace("cl","").replace("ml","").strip()
        s = re.sub(r"[^\d\.]", "", s)
        try:
            num = float(s)
            return int(num * 10) if num < 100 else int(num)
        except Exception:
            return np.nan

    def infer_grapes(row):
        wine_name = unidecode(str(row.get('wine', '')).lower().strip())
        producer  = unidecode(str(row.get('producer', '')).lower().strip())
        origin    = unidecode(str(row.get('origin', '')).lower().strip())
        combined  = f"{wine_name} {producer} {origin}"
        grape_keywords = {
            'nebbiolo':'Nebbiolo','tempranillo':'Tempranillo','cabernet sauvignon':'Cabernet Sauvignon',
            ' cabernet ':'Cabernet Sauvignon',' cab ':'Cabernet Sauvignon','merlot':'Merlot',
            'pinot noir':'Pinot Noir',' pinot ':'Pinot Noir','sangiovese':'Sangiovese','syrah':'Syrah',
            'shiraz':'Syrah','grenache':'Grenache','chardonnay':'Chardonnay',' chard ':'Chardonnay',
            'riesling':'Riesling','sauvignon blanc':'Sauvignon Blanc',' sauvignon ':'Sauvignon Blanc',
            ' sauv ':'Sauvignon Blanc','zinfandel':'Zinfandel','primitivo':'Primitivo','malbec':'Malbec',
            'grigio':'Pinot Grigio','garganega':'Garganega',"nero d avola":"Nero d'Avola",'barbera':'Barbera',
            'carmenere':'Carmenère','trebbiano':'Trebbiano','vermentino':'Vermentino','teroldego':'Teroldego'
        }
        hits = {g for k,g in grape_keywords.items() if k in f" {combined} "}
        if not hits:
            t = unidecode(str(row.get('type',''))).lower()
            if 'red' in t: hits.add('Red Blend')
            elif 'white' in t: hits.add('White Blend')
            elif 'sparkling' in t: hits.add('Sparkling Blend')
            elif 'rose' in t or 'rosé' in t: hits.add('Rosé Blend')
            else: hits.add('Unknown')
        return '/'.join(sorted(hits))

    def infer_body(row):
        c = str(row.get("color","")).lower()
        t = str(row.get("type","")).lower()
        if "sparkling" in t: return 2
        if "red" in c: return 4
        if "white" in c or "ros" in c: return 2
        return 3

    def infer_sweetness(row):
        t = str(row.get("type","")).lower()
        if "brut" in t or "dry" in t: return 1
        if "sweet" in t: return 5
        if "medium" in t: return 3
        return 3

    def infer_occasion(row):
        tier = row.get("price_tier")
        t = str(row.get("type","")).lower()
        if tier in ["Luxury","Ultra Luxury"]: return "Gifting"
        if "sparkling" in t or "ros" in t:     return "Celebration"
        if tier in ["Mid-range","Premium"]:    return "Dinner"
        return "Casual"

    # Ensure text cols exist to avoid "nan" literals
    for tcol in ["wine","producer","region","type","color","size","vintage"]:
        enhanced_df[tcol] = enhanced_df[tcol].fillna("").astype(str)

    enhanced_df["type_class"]     = enhanced_df.apply(infer_type_class, axis=1)
    enhanced_df["bottle_size_ml"] = enhanced_df["size"].apply(parse_bottle_size_ml)
    enhanced_df["grape_list"]     = enhanced_df.apply(infer_grapes, axis=1)
    enhanced_df["body"]           = enhanced_df.apply(infer_body, axis=1)
    enhanced_df["sweetness"]      = enhanced_df.apply(infer_sweetness, axis=1)
    enhanced_df["occasion"]       = enhanced_df.apply(infer_occasion, axis=1)

    # --- full_type: vectorized ---
    t = enhanced_df["type"].fillna("").astype(str).str.strip()
    c = enhanced_df["color"].fillna("").astype(str).str.strip()
    enhanced_df["full_type"] = (t.str.title() + (" " + c.str.title()).where(c.ne(""), "")).str.strip()

    # Stable dedupe key for scheduling logic if needed
    enhanced_df["wine_full_match_string"] = (
        enhanced_df["wine"].str.strip() + " " +
        enhanced_df["producer"].str.strip() + " " +
        enhanced_df["vintage"].astype(str).str.strip() + " " +
        enhanced_df["size"].astype(str).str.strip() + " " +
        enhanced_df["id"].astype(str).str.strip()
    ).str.replace(r"\s+", " ", regex=True).str.strip()

    # Final save
    final_stock_path = OUTPUT_PATH / "stock_df_final.pkl"
    enhanced_df.to_pickle(final_stock_path)

    stock_df = enhanced_df  # expose to later cells

    print(f"✅ Final wine dataset enriched and saved to: {final_stock_path}")
    try:
        from IPython.display import display
        display(enhanced_df.head(3))
    except Exception:
        print(enhanced_df.head(3).to_string(index=False))


📋 Stock columns (raw): ['ID', 'Stock', 'Minimum Order Q-ty', 'Size', 'Wine', 'Producer', 'Origin', 'Classification', 'Region', 'Country', 'Color', 'Type', 'Vintage', 'Rating', 'RP', 'WA', 'WS', 'JS', 'RG', 'VINOUS', 'JR', 'Decanter', 'NM', 'JA', 'AG', 'Falstaff', 'JD', 'DB', 'WInd', 'WCI', 'YB', 'AMA', 'JMQ', 'MDM', 'VINUM', 'AVG', 'EUR p/bt', 'CHF p/bt VAT excl.', 'CHF p/bt VAT incl.', 'OMT Last Offer Price CHF', 'OMT Last Offer Price Euro', 'OMT last offer date']
✅ Final wine dataset enriched and saved to: C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\stock_df_final.pkl


,id,stock,Minimum Order Q-ty,size,wine,producer,origin,classification,region,Country,...,Num_of_CM,region_group,type_class,bottle_size_ml,grape_list,body,sweetness,occasion,full_type,wine_full_match_string
0,40261,108,12,75.0,Bolaire,Château Bolaire,Bordeaux,NaN,Bordeaux,France,...,NaN,Bordeaux,Red,750,Unknown,4,3,Casual,Still Red,Bolaire Château Bolaire 2016 75.0 40261
1,54770,3,1,75.0,Ygrec de Ch. d'Yquem,Château d'Yquem,Bordeaux,NaN,Bordeaux,France,...,NaN,Bordeaux,White,750,Unknown,2,3,Dinner,Still White,Ygrec de Ch. d'Yquem Château d'Yquem 2020 75.0...
2,62182,96,6,75.0,Ygrec de Ch. d'Yquem,Château d'Yquem,Bordeaux,NaN,Bordeaux,France,...,22.0,Bordeaux,White,750,Unknown,2,3,Dinner,Still White,Ygrec de Ch. d'Yquem Château d'Yquem 2022 75.0...


In [5]:
# --- CELL 5: History (post-stock) — robust OMT parsing + de-dupe + UI artifacts ---
# Outputs:
#   - OUTPUT_PATH/history/wine_campaign_history.json         (FLAT map: id or wine::vintage -> {wine, vintage, last_campaign_date, emails_total})
#   - OUTPUT_PATH/campaign_index.json                        (by_id/by_name + emails + meta_by_id for UI hydration)
#   - OUTPUT_PATH/weekly_campaign_schedule_{YEAR}_week_{W}.json
#   - OUTPUT_PATH/locked_weeks/locked_calendar_{YEAR}_week_{W}.json
#   - OUTPUT_PATH/weekly_leads_{YEAR}_week_{W}.json
#   - OUTPUT_PATH/schedule_index.json

import os, io, re, time, shutil, json
from datetime import datetime
from pathlib import Path
from collections import defaultdict
import pandas as pd
import numpy as np

# ---------------- Bootstraps (if Cell 1 didn't run) ----------------
try:
    OUTPUT_PATH
    SOURCE_PATH
    TMP_PATH
except NameError:
    def _resolve_path(env_key, default_path_str): return Path(os.getenv(env_key, default_path_str))
    SOURCE_PATH = _resolve_path("INPUT_PATH",  str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "SOURCE_FILES"))
    OUTPUT_PATH = _resolve_path("OUTPUT_PATH", str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"))
    TMP_PATH    = OUTPUT_PATH / "_tmp"
    for p in (SOURCE_PATH, OUTPUT_PATH, TMP_PATH): p.mkdir(parents=True, exist_ok=True)

# ---------------- Settings ----------------
NUM_SLOTS = 5
DAYS_FULL = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
DAYMAP = {1:"Monday",2:"Tuesday",3:"Wednesday",4:"Thursday",5:"Friday",6:"Saturday",7:"Sunday"}

LOCK_HISTORY_CARDS = True         # keep history cards locked in weekly files
INCLUDE_SCHEDULED_AT = True       # include ISO timestamp of the campaign on each card

HISTORY_DIR = OUTPUT_PATH / "history"; HISTORY_DIR.mkdir(parents=True, exist_ok=True)
LOCKED_PATH  = OUTPUT_PATH / "locked_weeks"; LOCKED_PATH.mkdir(parents=True, exist_ok=True)
HISTORY_JSON = HISTORY_DIR / "wine_campaign_history.json"

# ---------------- Helpers ----------------
def _to_py(o):
    """JSON-safe conversion for numpy/pandas scalars."""
    if o is None: return None
    if isinstance(o, (np.integer,)):  return int(o)
    if isinstance(o, (np.floating,)):
        try:
            return None if np.isnan(o) else float(o)
        except Exception:
            return float(o)
    if isinstance(o, (np.bool_,)):    return bool(o)
    if isinstance(o, (pd.Timestamp,)):return o.isoformat()
    return o

CRITIC_COLS_CANON = ["avg_score"]  # if Cell 4 already computed/kept it
CRITIC_COLS_RAW = [
    "RP","WA","WS","JS","RG","VINOUS","JR","Decanter","NM","JA","AG","Falstaff","JD","DB",
    "WInd","WCI","YB","AMA","JMQ","MDM","VINUM","Rating"
]

def _compute_avg_score(row: pd.Series) -> float | None:
    """Robust critic average: use any available critic columns, ignore 0/NaN/out-of-range."""
    vals = []
    for col in CRITIC_COLS_CANON + [c for c in CRITIC_COLS_RAW if c in row.index]:
        v = pd.to_numeric(row.get(col), errors="coerce")
        if pd.notna(v) and 50 <= float(v) <= 100:
            vals.append(float(v))
    if not vals: 
        return None
    return round(float(np.mean(vals)), 1)

def _fallback_grapes(meta: dict) -> str:
    """Return a graceful fallback when grape_list is Unknown/blank."""
    g = (meta.get("grape_list") or "").strip().lower()
    if g and g != "unknown":
        return meta.get("grape_list")
    tc = (meta.get("type_class") or "").strip().lower()
    return {
        "red": "Red Blend",
        "white": "White Blend",
        "rose": "Rosé Blend",
        "rosé": "Rosé Blend",
        "sparkling": "Sparkling Blend",
        "dessert": "Dessert Blend",
    }.get(tc, "Blend")

# ---------------- Load stock meta from Cell 4 output ----------------
stock_df_path = OUTPUT_PATH / "stock_df_final.pkl"
stock_meta = {}
if stock_df_path.exists() and stock_df_path.stat().st_size > 0:
    _sdf = pd.read_pickle(stock_df_path)
    _sdf["id"] = _sdf["id"].astype(str).str.strip().str.replace(r"\.0$","",regex=True)

    # we'll compute a robust avg_score per row (ignoring 0s/outliers)
    have_cols = [c for c in (CRITIC_COLS_CANON + CRITIC_COLS_RAW) if c in _sdf.columns]
    for _, r in _sdf.dropna(subset=["id"]).drop_duplicates("id").iterrows():
        wid = (r["id"] or "").strip()
        if not wid: 
            continue

        # prefer a valid existing avg_score; else compute
        avg = pd.to_numeric(r.get("avg_score"), errors="coerce")
        avg = float(avg) if pd.notna(avg) and 50 <= float(avg) <= 100 else None
        if avg is None and have_cols:
            avg = _compute_avg_score(r)

        stock_meta[wid] = {
            "wine":            str(r.get("wine","")).strip(),
            "vintage":         str(r.get("vintage","NV")).strip() or "NV",
            "full_type":       str(r.get("full_type","")).strip(),
            "region_group":    str(r.get("region_group","")).strip(),
            "price_tier":      str(r.get("price_tier","")).strip(),
            "price_tier_key":  str(r.get("price_tier_key","")).strip(),
            "stock":           int(pd.to_numeric(r.get("stock"), errors="coerce") or 0),

            # extras for UI
            "producer":        str(r.get("producer","")).strip(),
            "chf_price":       (float(r["CHF Price"]) if pd.notna(r.get("CHF Price")) else None),
            "grape_list":      str(r.get("grape_list","")).strip(),
            "bottle_size_ml":  (int(r["bottle_size_ml"]) if pd.notna(r.get("bottle_size_ml")) else None),
            "type_class":      str(r.get("type_class","")).strip(),
            "body":            (_to_py(r.get("body")) if "body" in r else None),
            "sweetness":       (_to_py(r.get("sweetness")) if "sweetness" in r else None),
            "occasion":        str(r.get("occasion","")).strip(),
            "avg_score":       avg,   # ← fixed: recomputed and only present if valid
        }

# ---------------- Discover & safe-read OMT sources ----------------
def _find_omt_sources(base: Path) -> list[Path]:
    pats = ["*OMT*offer*.xlsx","*OMT*offer*.csv","*OMT*main*offer*.xlsx","*OMT*main*offer*.csv",
            "*campaign*list*.xlsx","*campaign*list*.csv","*offer*list*.xlsx","*offer*list*.csv"]
    out, seen = [], set()
    for pat in pats:
        for p in base.rglob(pat):
            if p.exists() and str(p) not in seen:
                seen.add(str(p)); out.append(p)
    return sorted(out)

def _safe_read_xlsx(p: Path) -> pd.DataFrame:
    for _ in range(3):
        try:
            with open(p,"rb") as f: buf=f.read()
            return pd.read_excel(io.BytesIO(buf), engine="openpyxl")
        except PermissionError: time.sleep(0.6)
        except Exception: break
    tmp = TMP_PATH / f"shadow_{int(time.time()*1000)}_{p.name}"
    try: shutil.copy2(p, tmp); return pd.read_excel(tmp, engine="openpyxl")
    except Exception: return pd.DataFrame()

def _safe_read_csv(p: Path) -> pd.DataFrame:
    for _ in range(3):
        try:
            with open(p,"rb") as f: buf=f.read()
            return pd.read_csv(io.BytesIO(buf))
        except PermissionError: time.sleep(0.6)
        except Exception: break
    tmp = TMP_PATH / f"shadow_{int(time.time()*1000)}_{p.name}"
    try: shutil.copy2(p, tmp); return pd.read_csv(tmp)
    except Exception: return pd.DataFrame()

def _load_omt(p: Path) -> pd.DataFrame:
    try:
        df = _safe_read_xlsx(p) if p.suffix.lower() in {".xlsx",".xls"} else _safe_read_csv(p)
        df.columns = [str(c).strip() for c in df.columns]
        return df
    except Exception:
        return pd.DataFrame()

# ---------------- Helpers for normalization & clustering ----------------
_SIZE_NUMS = {187,200,250,300,375,500,620,700,720,730,750,1000,1500,2000,3000,4500,6000,9000,12000}
_RE_SIZE_TOKEN = re.compile(r"(?i)\b(\d{3,5})\s*(ml|cl|l|lt|litre|liter)?\b")
_RE_SIZE_PACK  = re.compile(r"(?i)\b\d+\s*x\s*\d{3,5}\s*(ml|cl|l)\b")

def _strip_last_size_token(s: str) -> str:
    if not s: return ""
    parts = [p.strip() for p in str(s).split(",")]
    while len(parts) > 1:
        last = parts[-1]
        m = _RE_SIZE_TOKEN.search(last)
        only_num = last.replace(" ","").isdigit()
        ok_num = only_num and int(last.replace(" ","")) in _SIZE_NUMS
        if m or ok_num or _RE_SIZE_PACK.search(last): parts.pop()
        else: break
    return ", ".join(parts).strip()

def _norm(s): return re.sub(r"\s+"," ", str(s or "").strip())

def _extract_vintage(v, name=None, desc=None):
    v = _norm(v)
    if re.fullmatch(r"(19|20)\d{2}", v): return v
    for t in (name, desc):
        m = re.search(r"\b(19|20)\d{2}\b", str(t or ""))
        if m: return m.group(0)
    return "NV"

def _key_by_name(wine, vintage): 
    return f"{_norm(wine).lower()}::{_norm(vintage or 'NV').lower()}"

def _looks_like_wine_name(s: str) -> bool:
    s = _norm(s).lower()
    if not s: return False
    bad = ["accessor", "glass", "opener", "gift", "voucher", "packaging", "ship", "bag", "box"]
    return not any(b in s for b in bad)

def _unify_one(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return pd.DataFrame()
    cols = {c: re.sub(r"[^a-z0-9]+","_", c.strip().lower()) for c in df.columns}
    df = df.rename(columns=cols)

    def pick(*names): 
        for n in names:
            if n in df.columns: return n
        return None

    c_id   = pick("wine_id","id","item_no","item_no_","sku","product_id","code","item_code")
    c_name = pick("wine","wine_name","name","product_name","title","label","long_name","item_description","description")
    c_desc = pick("item_description","description")
    c_vint = pick("vintage","year")
    c_dt   = pick("schedule_datetime","scheduled_datetime","schedule_date_time","schedule_date","date","sent_at","created","execution_date","offer_date")
    c_sent = pick("number_of_sent_emails","emails_sent","sent_emails","emails","#_emails","mails_sent","sent")

    rows = []
    for _, r in df.iterrows():
        dt = pd.to_datetime(r.get(c_dt), errors="coerce", dayfirst=True, utc=False) if c_dt else pd.NaT
        if pd.isna(dt): 
            continue
        desc = str(r.get(c_desc) or "")
        raw_name = str(r.get(c_name) or "")
        name = _norm(raw_name) or _strip_last_size_token(desc)
        if not _looks_like_wine_name(name):
            continue
        vint = _extract_vintage(r.get(c_vint), raw_name, desc)
        wid  = _norm(r.get(c_id) or "").replace(".0","")
        if wid and not wid.isdigit():
            wid = ""  # name-key fallback for non-numeric ids
        emails = r.get(c_sent)
        try: emails = int(emails) if pd.notna(emails) else 0
        except: emails = 0

        rows.append({
            "schedule_dt": dt,
            "schedule_min": dt.floor("min"),
            "wine_id": wid,
            "wine_name": name,
            "vintage": vint,
            "item_core": _strip_last_size_token(desc or raw_name),
            "emails_sent": emails,
        })
    out = pd.DataFrame(rows)
    if out.empty: return out
    iso = out["schedule_dt"].dt.isocalendar()
    out["iso_year"] = iso.year.astype(int)
    out["iso_week"] = iso.week.astype(int)
    out["iso_wday"] = iso.day.astype(int)
    return out

# ---------------- Load & unify all OMT sources ----------------
files = _find_omt_sources(SOURCE_PATH)
frames = []
for p in files:
    dfp = _load_omt(p)
    if dfp is not None and not dfp.empty:
        frames.append(_unify_one(dfp))
raw = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
print(f"[HIST] OMT rows after unify (wine-only): {len(raw):,}")

if raw.empty:
    payload = {"by_id": {}, "by_name": {}, "emails_sent_by_id": {}, "emails_sent_by_name": {}, "meta_by_id": stock_meta}
    (OUTPUT_PATH / "campaign_index.json").write_text(json.dumps(payload, indent=2), encoding="utf-8")
    HISTORY_JSON.write_text(json.dumps({}, indent=2), encoding="utf-8")
    print("ℹ️ No OMT rows found. Wrote empty campaign_index & history.")
else:
    # ---------------- Cluster ----------------
    tmp = raw.copy()
    tmp["name_key"] = tmp.apply(lambda r: _key_by_name(r["wine_name"], r["vintage"]), axis=1)
    tmp["wine_key"] = tmp.apply(lambda r: r["wine_id"] if r["wine_id"] else r["name_key"], axis=1)

    gcols = ["schedule_min","wine_key","item_core","vintage","iso_year","iso_week","iso_wday"]
    cl = (tmp.groupby(gcols, dropna=False, as_index=False)
              .agg({"schedule_dt":"max","emails_sent":"sum","wine_id":"first","wine_name":"first"}))
    cl = (cl.sort_values(["schedule_min","wine_key","emails_sent"], ascending=[True,True,False])
            .drop_duplicates(["schedule_min","wine_key"], keep="first"))

    # ---------------- Last-campaign & emails totals ----------------
    last_by_id, last_by_name = {}, {}
    emails_by_id, emails_by_name = defaultdict(int), defaultdict(int)
    for _, r in cl.iterrows():
        wid = (r["wine_id"] or "").strip()
        nkey = _key_by_name(r["wine_name"], r["vintage"])
        dt_iso = pd.to_datetime(r["schedule_dt"]).date().isoformat()
        if wid:
            last_by_id[wid] = max(last_by_id.get(wid,""), dt_iso) if last_by_id.get(wid) else dt_iso
            emails_by_id[wid] += int(r.get("emails_sent") or 0)
        last_by_name[nkey] = max(last_by_name.get(nkey,""), dt_iso) if last_by_name.get(nkey) else dt_iso
        emails_by_name[nkey] += int(r.get("emails_sent") or 0)

    # ---------------- Flat history map ----------------
    history_flat = {}
    for _, r in cl.iterrows():
        wid = (r["wine_id"] or "").strip()
        nkey = _key_by_name(r["wine_name"], r["vintage"])
        key  = wid if wid else nkey
        if wid and not wid.isdigit():
            key = nkey
        history_flat[key] = {
            "wine": r["wine_name"],
            "vintage": r["vintage"],
            "last_campaign_date": (last_by_id.get(wid) if wid else last_by_name.get(nkey)) or "",
            "emails_total": int(emails_by_id.get(wid, 0) if wid else emails_by_name.get(nkey, 0)),
        }
    HISTORY_JSON.write_text(json.dumps(history_flat, indent=2), encoding="utf-8")

    # ---------------- campaign_index (for UI hydration) ----------------
    campaign_index = {
        "by_id": last_by_id,
        "by_name": last_by_name,
        "emails_sent_by_id": dict(emails_by_id),
        "emails_sent_by_name": dict(emails_by_name),
        "meta_by_id": stock_meta
    }
    (OUTPUT_PATH / "campaign_index.json").write_text(json.dumps(campaign_index, indent=2), encoding="utf-8")
    print("✅ history (flat) + campaign_index written.")

    # ---------------- Weekly schedules ----------------
    def _atomic_write(path: Path, text: str):
        tmp = path.with_suffix(path.suffix + ".tmp")
        tmp.write_text(text, encoding="utf-8")
        os.replace(tmp, path)

    idx_path = OUTPUT_PATH / "schedule_index.json"
    try: schedule_idx = json.loads(idx_path.read_text(encoding="utf-8")) if idx_path.exists() else {}
    except Exception: schedule_idx = {}

    latest_y = latest_w = None

    wk = (cl.sort_values(["iso_year","iso_week","wine_key","schedule_dt"])
            .groupby(["iso_year","iso_week","wine_key"], as_index=False)
            .agg({"schedule_dt":"max","emails_sent":"sum","wine_id":"first","wine_name":"first","vintage":"first","iso_wday":"first"}))

    for (yr, w), sub in wk.groupby(["iso_year","iso_week"]):
        cal = {d: [] for d in DAYS_FULL}
        seen = set()
        for r in sub.sort_values(["schedule_dt","emails_sent"], ascending=[True,False]).to_dict("records"):
            dname = DAYMAP.get(int(r.get("iso_wday",1)), "Monday")
            key   = r["wine_key"]
            if key in seen: 
                continue
            wid   = (r.get("wine_id") or "").strip()
            meta  = stock_meta.get(wid, {})

            label = (r.get("wine_name") or "").strip() or (r.get("item_core") or "").strip() or meta.get("wine") or "Unknown"
            vint  = (r.get("vintage") or "").strip() or meta.get("vintage") or "NV"
            sched_date = pd.to_datetime(r["schedule_dt"])

            # --- build extras with grape fallback & fixed avg_score ---
            extras = {
                "producer":       _to_py(meta.get("producer")),
                "chf_price":      _to_py(meta.get("chf_price")),
                "grape_list":     _fallback_grapes(meta),  # ← never 'Unknown'
                "bottle_size_ml": _to_py(meta.get("bottle_size_ml")),
                "type_class":     _to_py(meta.get("type_class")),
                "body":           _to_py(meta.get("body")),
                "sweetness":      _to_py(meta.get("sweetness")),
                "occasion":       _to_py(meta.get("occasion")),
                "avg_score":      _to_py(meta.get("avg_score")),  # ← recomputed if possible
            }

            item = {
                "id": wid,
                "wine": label,
                "vintage": vint,
                "full_type": meta.get("full_type","") or "Unknown",
                "region_group": meta.get("region_group","") or "Unknown",
                "price_tier": meta.get("price_tier","") or "",
                "stock": int(meta.get("stock", 0)),
                "match_quality": "History (Locked)",
                "avg_cpi_score": 0.0,
                "locked": bool(LOCK_HISTORY_CARDS),
                "emails_sent": int(r.get("emails_sent") or 0),
                "last_campaign_date": sched_date.date().isoformat(),

                # convenience / back-compat
                "name": label,
                "price_tier_id": meta.get("price_tier_key",""),
                "extras": extras
            }
            if INCLUDE_SCHEDULED_AT:
                item["scheduled_at"] = sched_date.isoformat()

            if len(cal[dname]) < NUM_SLOTS:
                cal[dname].append(item)
                seen.add(key)

        # year-week schedule
        out_year = OUTPUT_PATH / f"weekly_campaign_schedule_{yr}_week_{w}.json"
        _atomic_write(out_year, json.dumps(cal, indent=2))

        # locked snapshot
        out_locked = LOCKED_PATH / f"locked_calendar_{yr}_week_{w}.json"
        locked_week = {day: [(cal[day][i] if i < len(cal[day]) else None) for i in range(NUM_SLOTS)] for day in DAYS_FULL}
        _atomic_write(out_locked, json.dumps(locked_week, indent=2))

        # empty leads buckets for UI
        leads_json = OUTPUT_PATH / f"weekly_leads_{yr}_week_{w}.json"
        _atomic_write(leads_json, json.dumps({"TueWed": [], "ThuFri": []}, indent=2))

        # index bookkeeping
        schedule_idx.setdefault(str(yr), {})[str(w)] = {
            "json": out_year.name,
            "updated_at": datetime.now().isoformat(timespec="seconds")
        }
        if (latest_y is None) or (yr > latest_y) or (yr == latest_y and w > (latest_w or 0)):
            latest_y, latest_w = yr, w

    if latest_y is not None:
        schedule_idx["_latest_year"] = str(latest_y)
        schedule_idx["_latest_week"] = str(latest_w)

    _atomic_write(idx_path, json.dumps(schedule_idx, indent=2))
    print("📅 weekly schedules & 🔒 locked calendars written (post-stock).")

# ---------------- Quick sanity (optional) ----------------
try:
    d = json.loads(HISTORY_JSON.read_text(encoding="utf-8"))
    bad = [k for k in d.keys() if not (re.fullmatch(r"\d+", k) or "::" in k)]
    if bad:
        print("❌ Bad keys in history:", bad[:10])
    else:
        print("✅ History keys look good (id or wine::vintage).")
except Exception as e:
    print("⚠️ History sanity skipped:", e)


[HIST] OMT rows after unify (wine-only): 25,176
✅ history (flat) + campaign_index written.
📅 weekly schedules & 🔒 locked calendars written (post-stock).
✅ History keys look good (id or wine::vintage).


In [6]:
# --- CELL 6: CPI Calculation + Filter Wiring (Clean & Robust) ---
# Honors UI filters (loyalty, wine type, bottle size, price tier, last stock, seasonality),
# computes CPI against stock, and saves compact outputs the UI can reuse.

import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
from time import perf_counter

# ---------- Helpers ----------
def get_season_from_week(week_no: int):
    if 1 <= week_no <= 8 or 49 <= week_no <= 53: return "Winter"
    if 9  <= week_no <= 22: return "Spring"
    if 23 <= week_no <= 35: return "Summer"
    if 36 <= week_no <= 48: return "Autumn"
    return "Unknown"

def extract_ids_from_weekly_calendar(obj) -> set:
    ids = set()
    if isinstance(obj, dict):
        for arr in obj.values():
            if isinstance(arr, list):
                for it in arr:
                    if isinstance(it, dict):
                        v = it.get("id") or it.get("wine_id")
                        if v is not None and str(v).strip():
                            ids.add(str(v))
    elif isinstance(obj, list):
        for it in obj:
            if isinstance(it, dict):
                v = it.get("id") or it.get("wine_id")
                if v is not None and str(v).strip():
                    ids.add(str(v))
    return ids

def coerce_numeric(s, default=np.nan):
    try:
        return pd.to_numeric(s, errors="coerce")
    except Exception:
        return default

# keep the calendar_year chosen earlier; do NOT reset to now().year later
try:
    week_number = int(week_number)
except Exception:
    week_number = int(os.getenv("WEEK_NUMBER", datetime.now().isocalendar().week))
try:
    year = int(calendar_year)
except Exception:
    year = datetime.now().year

selected_season = get_season_from_week(week_number)

# text-mode tqdm (avoid widget deps)
try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **k): return x

# ---------- Ensure inputs are present ----------
# If notebook was run standalone, try loading from previous cell outputs
if 'clients_df' not in globals():
    p = OUTPUT_PATH / "filtered_clients.pkl"
    clients_df = pd.read_pickle(p) if p.exists() else pd.DataFrame()
if 'stock_df' not in globals():
    p = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df = pd.read_pickle(p) if p.exists() else pd.DataFrame()

# Normalize schemas
clients_df = clients_df.copy()
clients_df.columns = (
    clients_df.columns
    .str.strip().str.lower().str.replace(" ", "_").str.replace(".", "", regex=False)
)
if 'customer_no' in clients_df.columns:
    clients_df['customer_no'] = clients_df['customer_no'].astype(str).str.strip()
if 'item_no' in clients_df.columns:
    clients_df['item_no'] = clients_df['item_no'].astype(str).str.strip()

stock_df = stock_df.copy()
if 'id' in stock_df.columns:
    stock_df['id'] = stock_df['id'].astype(str).str.strip()

# critic score column consistency
if 'avg_score' not in stock_df.columns and 'AVG' in stock_df.columns:
    stock_df.rename(columns={'AVG': 'avg_score'}, inplace=True)

# ---------- Loyalty derivation (counts → tier) ----------
if {'customer_no','item_no'}.issubset(clients_df.columns):
    purchase_counts = clients_df.groupby('customer_no')['item_no'].count().reset_index(name='purchase_count')
    purchase_counts['loyalty_level'] = purchase_counts['purchase_count'].apply(
        lambda x: 'vip' if x >= 20 else 'gold' if x >= 10 else 'silver' if x >= 5 else 'bronze'
    )
    clients_df = clients_df.drop(columns=['loyalty_level'], errors='ignore') \
                           .merge(purchase_counts[['customer_no','loyalty_level']], on='customer_no', how='left')
else:
    clients_df['loyalty_level'] = 'bronze'
clients_df['loyalty_level'] = clients_df['loyalty_level'].fillna('bronze')

# ---------- Bottle size ML on clients (fallback only) ----------
if 'bottle_size' in clients_df.columns:
    def _to_ml(x):
        try:
            v = float(x)
            return round(v*10) if v < 100 else v
        except:
            return np.nan
    clients_df['bottle_size_ml'] = clients_df['bottle_size'].apply(_to_ml)
else:
    clients_df['bottle_size_ml'] = np.nan

# Optional sales date
sales_dates = clients_df[['customer_no','sales_date']].drop_duplicates() \
    if 'sales_date' in clients_df.columns else pd.DataFrame(columns=['customer_no','sales_date'])

# Vintage from stock by item_no (if available)
if 'item_no' in clients_df.columns and 'vintage' in stock_df.columns and 'id' in stock_df.columns:
    stock_vintage_map = stock_df.set_index('id')['vintage'].astype(str).to_dict()
    clients_df['vintage'] = clients_df['item_no'].map(stock_vintage_map)
else:
    clients_df['vintage'] = 'Unknown'

def classify_vintage_group(vint):
    try:
        s = str(vint).strip().upper()
        if s == "NV": return "Non-Vintage Lover"
        y = int(s); cy = datetime.now().year
        if y == cy: return "Likes Current Vintage"
        if y == cy-1: return "Likes En Primeur"
        if cy - y <= 3: return "Likes Young Wines"
        if cy - y <= 8: return "Likes Mature Wines"
        if cy - y > 15: return "Likes Old Wines"
        return "Unknown"
    except:
        return "Unknown"

clients_df['inferred_vintage'] = clients_df['vintage'].apply(classify_vintage_group)

def classify_size_from_ml(size_ml):
    try:
        v = int(float(size_ml))
        m = {375:"Half",750:"Standard",1500:"Magnum",3000:"Jeroboam",4500:"Rehoboam",
             6000:"Methuselah",9000:"Salmanazar",12000:"Balthazar",15000:"Nebuchadnezzar",
             18000:"Melchior",27000:"Primat"}
        return m.get(v, "Unknown")
    except:
        return "Unknown"

# ---------- Build purchases_df for preference inference ----------
needed = ['id','grape_list','type','region','sweetness','body','price_tier','avg_score','vintage','bottle_size_ml']
stock_merge_df = stock_df.copy()
for c in needed:
    if c not in stock_merge_df.columns:
        stock_merge_df[c] = np.nan

if 'item_no' not in clients_df.columns:
    purchases_df = clients_df.copy()
    for c in ['grape_list','type','region','sweetness','body','price_tier','avg_score','vintage','bottle_size_ml']:
        purchases_df[c] = np.nan
else:
    purchases_df = clients_df.merge(
        stock_merge_df[needed], left_on='item_no', right_on='id', how='left'
    )

purchases_df['vintage'] = purchases_df.get('vintage', pd.Series(["Unknown"]*len(purchases_df))).fillna("Unknown")
if 'bottle_size_ml' not in purchases_df.columns:
    purchases_df['bottle_size_ml'] = 750
purchases_df['size_group'] = purchases_df['bottle_size_ml'].apply(classify_size_from_ml)

# Occasion inference (coarse)
def infer_occasion(row):
    try:
        price = str(row.get('price_tier','')).lower()
        wine_type = str(row.get('type','')).lower()
        size = float(row.get('bottle_size_ml',750))
        score = float(row.get('avg_score',0))
        sweetness = float(row.get('sweetness',0))
        if size >= 3000: return 'Big Event'
        if size >= 1500: return 'Celebration'
        if 'sparkling' in wine_type or sweetness >= 4: return 'Celebration'
        if score >= 92 or (price in ['luxury','ultra luxury','premium'] and size == 750): return 'Gift'
        if price in ['budget','entry'] and size <= 750 and sweetness <= 2: return 'Everyday'
        if price in ['mid-range','premium'] and 'still' in wine_type: return 'Dinner'
        if score >= 90 and price in ['mid-range','premium']: return 'Dinner'
        if sweetness <= 2 and score >= 88 and size <= 750: return 'Everyday'
        if price in ['mid-range','premium','luxury'] and 'still' in wine_type: return 'Personal Consumption'
        return 'Unknown'
    except:
        return 'Unknown'
purchases_df['occasion'] = purchases_df.apply(infer_occasion, axis=1)

# ---------- Preference aggregation ----------
def split_and_flatten(series, delimiter='/'):
    return [x.strip() for s in series.dropna() for x in str(s).split(delimiter) if x.strip()]

inferred_pref = purchases_df.groupby('customer_no').agg({
    'grape_list':  lambda x: ','.join(sorted(set(split_and_flatten(x)))),
    'type':        lambda x: ','.join(sorted(set(x.dropna().astype(str).str.lower()))),
    'region':      lambda x: ','.join(sorted(set(x.dropna().astype(str).str.lower()))),
    'sweetness':   lambda x: round(coerce_numeric(x).dropna().astype(float).mean(), 2) if len(x.dropna()) else np.nan,
    'body':        lambda x: round(coerce_numeric(x).dropna().astype(float).mean(), 2) if len(x.dropna()) else np.nan,
    'price_tier':  lambda x: x.dropna().mode().iloc[0] if not x.dropna().mode().empty else '',
    'avg_score':   lambda x: round(coerce_numeric(x).dropna().astype(float).mean(), 2) if len(x.dropna()) else np.nan,
    'vintage':     lambda x: ','.join(sorted(set(x.dropna().astype(str)))),
    'size_group':  lambda x: ','.join(sorted(set(x.dropna().astype(str)))),
    'occasion':    lambda x: ','.join(sorted(set(x.dropna().astype(str))))
}).reset_index().rename(columns={
    'grape_list':'inferred_grape_preferences',
    'type':'inferred_type',
    'region':'inferred_region',
    'sweetness':'inferred_sweetness',
    'body':'inferred_body',
    'price_tier':'inferred_budget',
    'avg_score':'avg_critic_score',
    'vintage':'inferred_vintage',
    'size_group':'inferred_size',
    'occasion':'inferred_occasion'
})

# Fallback vintage/size preferences when inference is missing
clients_df['fallback_size_label'] = clients_df['bottle_size_ml'].apply(classify_size_from_ml)
fallback_vintage = (
    clients_df.groupby('customer_no')['vintage']
    .agg(lambda x: x.dropna().mode().iloc[0] if not x.dropna().mode().empty else 'Unknown')
    .reset_index(name='fallback_vintage')
)
fallback_vintage['fallback_vintage_label'] = fallback_vintage['fallback_vintage'].apply(classify_vintage_group)

inferred_pref = inferred_pref.merge(
    fallback_vintage[['customer_no','fallback_vintage_label']], on='customer_no', how='left'
)
inferred_pref['inferred_vintage'] = inferred_pref['fallback_vintage_label'].fillna(inferred_pref['inferred_vintage'])
inferred_pref.drop(columns=['fallback_vintage_label'], inplace=True)

inferred_pref = inferred_pref.merge(
    clients_df[['customer_no','fallback_size_label']].drop_duplicates(),
    on='customer_no', how='left'
)
inferred_pref['inferred_size'] = inferred_pref['fallback_size_label'].fillna(inferred_pref['inferred_size'])
inferred_pref.drop(columns=['fallback_size_label'], inplace=True)

# Merge traits + loyalty + sales date
client_pref_df = (inferred_pref
    .merge(clients_df[['customer_no']].drop_duplicates(), on='customer_no', how='right')
    .merge(clients_df[['customer_no','loyalty_level']].drop_duplicates(), on='customer_no', how='left')
    .merge(sales_dates, on='customer_no', how='left')
)
client_pref_df['loyalty_level'] = client_pref_df['loyalty_level'].fillna('bronze')

# Flags
client_pref_df['prefers_high_scores'] = (coerce_numeric(client_pref_df['avg_critic_score']).fillna(0) >= 95)
stock_df['avg_score'] = coerce_numeric(stock_df.get('avg_score', np.nan))
stock_df['high_score'] = stock_df['avg_score'].ge(95)

# ---------- UI Filters (from Cell 1 or env) ----------
if 'filters' not in globals():
    try:
        filters = json.loads(os.getenv("FILTER_INPUTS", "{}"))
    except Exception:
        filters = {}

# Loyalty filter (client subset)
loy = str(filters.get('loyalty','all') or 'all').lower()
if loy != 'all':
    before = len(client_pref_df)
    client_pref_df = client_pref_df.loc[client_pref_df['loyalty_level'].str.lower() == loy].copy()
    after = len(client_pref_df)
    print(f"🎯 Loyalty filter [{loy}] → clients: {before} → {after}")

# Wine type filter (stock subset)
wt = (filters.get('wine_type') or '').strip().lower()
if wt:
    if wt in {'red','white','rose','rosé','sparkling','dessert'}:
        if wt in {'rose','rosé'}:
            mask = stock_df.get('color','').astype(str).str.contains('ros', case=False, na=False)
        elif wt == 'sparkling':
            mask = stock_df.get('type','').astype(str).str.contains('sparkling', case=False, na=False) \
                   | stock_df.get('color','').astype(str).str.contains('spark', case=False, na=False)
        elif wt == 'dessert':
            mask = stock_df.get('type','').astype(str).str.contains('dessert', case=False, na=False)
        else:
            mask = stock_df.get('color','').astype(str).str.contains(wt, case=False, na=False)
        before = len(stock_df)
        stock_df = stock_df.loc[mask].copy()
        after = len(stock_df)
        print(f"🍷 Wine type [{wt}] → wines: {before} → {after}")

# Bottle size filter
bs = filters.get('bottle_size', None)
try:
    bs_ml = int(float(bs)) if bs is not None else None
except Exception:
    bs_ml = None
if bs_ml:
    before = len(stock_df)
    stock_df = stock_df.loc[(coerce_numeric(stock_df.get('bottle_size_ml')).round(0) == bs_ml)].copy()
    after = len(stock_df)
    print(f"🍾 Bottle size [{bs_ml}ml] → wines: {before} → {after}")

# Price tier bucket filter
pt = (filters.get('price_tier_bucket') or '').strip()
if pt:
    before = len(stock_df)
    stock_df = stock_df.loc[stock_df.get('price_tier','').astype(str).str.lower() == pt.lower()].copy()
    after = len(stock_df)
    print(f"💰 Price tier [{pt}] → wines: {before} → {after}")

# Low stock filter
if filters.get('last_stock', False):
    th = int(filters.get('last_stock_threshold', 10) or 10)
    before = len(stock_df)
    stock_df = stock_df.loc[coerce_numeric(stock_df.get('stock')).fillna(0) < th].copy()
    after = len(stock_df)
    print(f"⚠️ Last stock < {th} → wines: {before} → {after}")

# Seasonality filter (only if column exists)
if filters.get("seasonality_boost", False) and "seasonality_boost" in stock_df.columns:
    stock_df["seasonal_match"] = stock_df["seasonality_boost"].apply(
        lambda x: selected_season in x if isinstance(x, list) else False
    )
    before = len(stock_df)
    stock_df = stock_df.loc[stock_df["seasonal_match"] == True].copy()
    after = len(stock_df)
    print(f"🌿 Seasonality [{selected_season}] → wines: {before} → {after}")
elif filters.get("seasonality_boost", False):
    print("⚠️ 'seasonality_boost' column not found — skipping seasonality filter")

# Recency filter (avoid using items that were already used last year in same week, or this week if already emitted)
def _load_calendar_ids(p: Path) -> set:
    try:
        if p.suffix.lower() == ".pkl":
            obj = pd.read_pickle(p)
            cal = obj.get("weekly_calendar", obj) if isinstance(obj, dict) else obj
        else:
            cal = json.loads(p.read_text(encoding="utf-8"))
            cal = cal.get("weekly_calendar", cal)
        return extract_ids_from_weekly_calendar(cal)
    except Exception:
        return set()

candidates = [
    OUTPUT_PATH / f"weekly_campaign_schedule_{year-1}_week_{week_number}.pkl",
    OUTPUT_PATH / f"weekly_campaign_schedule_{year-1}_week_{week_number}.json",
    OUTPUT_PATH / f"weekly_campaign_schedule_{year}_week_{week_number}.pkl",
    OUTPUT_PATH / f"weekly_campaign_schedule_{year}_week_{week_number}.json",
    OUTPUT_PATH / f"weekly_campaign_schedule_week_{week_number}.pkl",   # legacy
    OUTPUT_PATH / f"weekly_campaign_schedule_week_{week_number}.json",   # legacy
]
past_ids = set()
used_path = None
for p in candidates:
    if p.exists():
        ids = _load_calendar_ids(p)
        if ids:
            past_ids = ids
            used_path = p
            break

if past_ids:
    before = len(stock_df)
    stock_df = stock_df.loc[~stock_df['id'].astype(str).isin(past_ids)].copy()
    after = len(stock_df)
    print(f"⏮️ Recency filter → removed {before - after} (source: {used_path.name})")
else:
    print(f"📁 No past schedule for Week {week_number} → skipping recency filter")

# Refresh flags after filters
stock_df['avg_score'] = coerce_numeric(stock_df.get('avg_score'))
stock_df['high_score'] = stock_df['avg_score'].ge(95)

# ---------- CPI compute ----------
display_col = 'wine' if 'wine' in stock_df.columns else 'id'

def compute_cpi_matrix(client_df, stock_df, style="default", display_col='wine'):
    stock_df = stock_df.copy()
    stock_df['id'] = stock_df['id'].astype(str)

    # Baseline weights
    weights = {
        'grape': 1.0, 'type': 1.0, 'region': 1.0,
        'sweetness': 0.5, 'body': 0.5,
        'budget': 0.75, 'prefers_high_scores': 0.75,
        'avg_score': 0.5
    }
    loyalty_bonus = {'bronze': 0.0, 'silver': 0.25, 'gold': 0.5, 'vip': 0.75}

    # Persona tweaks
    st = (style or 'default').lower()
    if st == "cat":
        weights['type'] = 1.2
        weights['region'] = 1.2
        weights['avg_score'] = 0.4
    elif st == "nigo":
        weights['budget'] = 1.0
        weights['avg_score'] = 0.7

    total_possible = sum(weights.values()) + max(loyalty_bonus.values())

    # Ensure numeric for tolerance checks
    sd_sweet = coerce_numeric(stock_df.get('sweetness')).fillna(-999)
    sd_body  = coerce_numeric(stock_df.get('body')).fillna(-999)

    result = []
    it = client_df.iterrows() if len(client_df) else iter([(-1, {'customer_no':'GLOBAL','inferred_grape_preferences':'','inferred_type':'','inferred_region':'','inferred_sweetness':np.nan,'inferred_body':np.nan,'inferred_budget':'','avg_critic_score':np.nan,'loyalty_level':'bronze','prefers_high_scores':False})])

    for _, client in tqdm(it, total=max(len(client_df),1), desc="🔄 Generating CPI vectors"):
        score = pd.Series(0.0, index=stock_df.index)

        # grapes
        grape_prefs = set(str(client.get('inferred_grape_preferences','')).lower().split(','))
        wine_grapes = stock_df.get('grape_list','').fillna('').astype(str).str.lower().str.split('/')
        score += wine_grapes.apply(lambda gs: any(g.strip() in grape_prefs for g in gs if g)).astype(float) * weights['grape']

        # type/region
        score += (stock_df.get('type','').fillna('').astype(str).str.lower()
                  .str.contains(str(client.get('inferred_type','')).lower(), na=False)) * weights['type']
        score += (stock_df.get('region','').fillna('').astype(str).str.lower()
                  .str.contains(str(client.get('inferred_region','')).lower(), na=False)) * weights['region']

        # sweetness/body (tolerance)
        cs = client.get('inferred_sweetness', np.nan)
        cb = client.get('inferred_body', np.nan)
        if pd.notna(cs):
            score += (np.isclose(sd_sweet, float(cs), atol=0.5)).astype(float) * weights['sweetness']
        if pd.notna(cb):
            score += (np.isclose(sd_body,  float(cb), atol=0.5)).astype(float) * weights['body']

        # budget match
        score += (stock_df.get('price_tier','').fillna('').astype(str).str.lower()
                  == str(client.get('inferred_budget','')).lower()) * weights['budget']

        # high score preference
        pref_hi = bool(client.get('prefers_high_scores', False))
        score += (stock_df['high_score'] & pref_hi).astype(float) * weights['prefers_high_scores']

        # general avg score quality
        score += (coerce_numeric(stock_df['avg_score']).fillna(0) >= 90).astype(float) * weights['avg_score']

        # loyalty
        score += loyalty_bonus.get(str(client.get('loyalty_level','bronze')).lower(), 0)

        score /= total_possible
        result.append(score.round(4).rename(f"pref_cpi_for_{client.get('customer_no','GLOBAL')}"))

    matrix = pd.concat([stock_df[['id', display_col]].reset_index(drop=True)] + result, axis=1)
    return matrix

style = (filters.get('style') or 'default').lower()
t0 = perf_counter()
cpi_matrix = compute_cpi_matrix(client_pref_df, stock_df, style=style, display_col=display_col)
print("⏱️ CPI computation completed in", round(perf_counter() - t0, 2), "seconds.")

# ---------- Attach average CPI per wine BEFORE saving UI snapshot ----------
cpi_cols = [c for c in cpi_matrix.columns if c.startswith("pref_cpi_for_")]
if cpi_cols:
    cpi_avg_df = pd.DataFrame({
        "id": cpi_matrix["id"].astype(str),
        "avg_cpi_score": cpi_matrix[cpi_cols].mean(axis=1).round(4)
    })
    stock_df = stock_df.merge(cpi_avg_df, on="id", how="left")
else:
    stock_df["avg_cpi_score"] = np.nan

# ---------- Save outputs ----------
client_pref_df.to_pickle(OUTPUT_PATH / "client_pref_df_latest.pkl")
cpi_matrix.to_pickle(OUTPUT_PATH / "cpi_matrix_latest.pkl")

# a compact stock file the webapp can use to render cards
ui_cols = [
    'id','wine','producer','vintage','price_tier','stock','full_type','type','color',
    'region','region_group','bottle_size_ml','avg_score','high_score','avg_cpi_score'
]
present = [c for c in ui_cols if c in stock_df.columns]
stock_df[present].to_pickle(OUTPUT_PATH / "stock_for_ui_latest.pkl")

print("✅ Preferences, CPI matrix, and UI stock snapshot saved (.pkl).")
print("🧪 CPI Matrix shape:", cpi_matrix.shape, "| UI stock cols:", present)


🍾 Bottle size l] → wines: 4578 → 2405
⏮️ Recency filter → removed 16 (source: weekly_campaign_schedule_2024_week_38.json)


🔄 Generating CPI vectors: 100%|█████████████████████████████████████████████████████| 345/345 [00:04<00:00, 75.55it/s]


⏱️ CPI computation completed in 4.76 seconds.
✅ Preferences, CPI matrix, and UI stock snapshot saved (.pkl).
🧪 CPI Matrix shape: (3719, 347) | UI stock cols: ['id', 'wine', 'producer', 'vintage', 'price_tier', 'stock', 'full_type', 'type', 'color', 'region', 'region_group', 'bottle_size_ml', 'avg_score', 'high_score', 'avg_cpi_score']


In [7]:
# --- CELL 7: Build/refresh fallback_pool safely ---
# Goal: give the scheduler a good "pool" to draw from when client prefs can't fill the week.

import os, json
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

# ---------- Fallbacks if prior cells didn't run ----------
try:
    OUTPUT_PATH
except NameError:
    _default_output = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
    OUTPUT_PATH = Path(os.getenv("OUTPUT_PATH", _default_output))
    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

if 'stock_df' not in globals():
    _stock_pkl = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df = pd.read_pickle(_stock_pkl) if _stock_pkl.exists() else pd.DataFrame()

if 'id' not in stock_df.columns:
    stock_df['id'] = ""  # ensure merge key exists

# ---------- Guards: ensure filters + UI vars exist even if Cell 5/6 didn't run ----------
# 1) Ensure `filters` is a dict (load from file/env if missing)
if 'filters' not in globals() or not isinstance(filters, dict):
    filters = {}
    _filters_path = os.path.join("notebooks", "filters.json")
    if os.path.exists(_filters_path):
        try:
            with open(_filters_path, "r", encoding="utf-8") as f:
                filters = json.load(f) or {}
        except Exception as _e:
            print(f"⚠️ Could not read filters.json: {_e}")
    else:
        try:
            env_f = os.getenv("FILTER_INPUTS", "")
            if env_f:
                filters = json.loads(env_f)
        except Exception:
            pass

# 2) Define UI variables with robust defaults
if 'selected_type' not in globals():
    selected_type = filters.get("wine_type", None)

if 'selected_size' not in globals():
    _sz = filters.get("bottle_size", None)
    try:
        selected_size = int(_sz) if _sz is not None and str(_sz).strip().lower() != "bigger" else None
    except Exception:
        selected_size = None

if 'last_stock' not in globals():
    _ls = filters.get("last_stock", False)
    last_stock = (str(_ls).strip().lower() in ("1", "true", "yes", "y"))

if 'seasonality_boost' not in globals():
    _sb = filters.get("seasonality_boost", False)
    seasonality_boost = (str(_sb).strip().lower() in ("1", "true", "yes", "y"))

def _parse_size_to_ml(val):
    try:
        s = str(val).strip().lower().replace("ml","").replace("cl","").replace("l","")
        if not s: return np.nan
        v = float(s)
        if v <= 100:  # cl
            return int(round(v * 10))
        if v < 20:    # liters
            return int(round(v * 1000))
        return int(round(v))  # already ml
    except Exception:
        return np.nan

# ---------- Try CPI-augmented pool; else fall back to stock only ----------
try:
    if "merged_cpi_df" in globals():
        _cpi_src = merged_cpi_df.copy()
        cpi_cols = [c for c in _cpi_src.columns if c.startswith("pref_cpi_for_")]
        _avg = _cpi_src[cpi_cols].mean(axis=1) if cpi_cols else pd.Series(np.nan, index=_cpi_src.index)
        cpi_avg_df = _cpi_src[['id']].copy()
        cpi_avg_df['avg_cpi'] = _avg
    else:
        _cpi_path = OUTPUT_PATH / "cpi_matrix_latest.pkl"
        _cpi = pd.read_pickle(_cpi_path) if _cpi_path.exists() else pd.DataFrame()
        if _cpi.empty or 'id' not in _cpi.columns:
            raise FileNotFoundError("cpi_matrix_latest.pkl missing or malformed")
        cpi_cols = [c for c in _cpi.columns if c.startswith("pref_cpi_for_")]
        _avg = _cpi[cpi_cols].mean(axis=1) if cpi_cols else pd.Series(np.nan, index=_cpi.index)
        cpi_avg_df = pd.DataFrame({"id": _cpi['id'].astype(str), "avg_cpi": _avg.values})

    # Assemble base pool from stock with useful columns
    _stock_cols = ['id','wine','vintage','stock','avg_score','price_tier','full_type','region_group']
    have_cols = [c for c in _stock_cols if c in stock_df.columns]
    base = stock_df[have_cols].copy()
    base['id'] = base['id'].astype(str)

    # Bottle size to ML (prefer existing column)
    if 'bottle_size_ml' in stock_df.columns:
        base = base.merge(stock_df[['id','bottle_size_ml']], on='id', how='left')
    else:
        size_sources = [c for c in stock_df.columns if c.lower() in ('size','size_cl','bottle_size','bottle size')]
        if size_sources:
            _sz = stock_df[['id', size_sources[0]]].rename(columns={size_sources[0]: 'size_raw'})
            _sz['bottle_size_ml'] = _sz['size_raw'].apply(_parse_size_to_ml)
            base = base.merge(_sz[['id','bottle_size_ml']], on='id', how='left')
        else:
            base['bottle_size_ml'] = np.nan

    # Attach CPI average
    fallback_pool = base.merge(cpi_avg_df, on='id', how='left')

except Exception as e:
    print(f"⚠️ Could not build CPI-based fallback pool: {e}")
    base_cols = ['id','wine','vintage','stock','avg_score','price_tier','full_type','region_group']
    have_cols = [c for c in base_cols if c in stock_df.columns]
    if 'id' not in have_cols:
        have_cols = ['id'] + have_cols
    fallback_pool = stock_df[have_cols].copy()
    if 'bottle_size_ml' in stock_df.columns and 'bottle_size_ml' not in fallback_pool.columns:
        fallback_pool = fallback_pool.merge(stock_df[['id','bottle_size_ml']], on='id', how='left')
    fallback_pool['avg_cpi'] = np.nan  # ensure column exists

# ---------- Ensure dtypes ----------
fallback_pool['stock']     = pd.to_numeric(fallback_pool.get('stock', 0), errors='coerce').fillna(0).astype(int)
fallback_pool['avg_cpi']   = pd.to_numeric(fallback_pool.get('avg_cpi', np.nan), errors='coerce')
fallback_pool['avg_score'] = pd.to_numeric(fallback_pool.get('avg_score', np.nan), errors='coerce')

# If CPI avg came from Cell 6 as avg_cpi_score, use it
if ('avg_cpi' not in fallback_pool.columns) or fallback_pool['avg_cpi'].isna().all():
    if 'avg_cpi_score' in stock_df.columns:
        try:
            fallback_pool = fallback_pool.merge(
                stock_df[['id','avg_cpi_score']].rename(columns={'avg_cpi_score':'avg_cpi'}),
                on='id', how='left'
            )
        except Exception:
            pass

# ---------- Apply UI filters ----------
_selected_type = (selected_type or "").strip()
_selected_size = selected_size
_last_stock    = bool(last_stock)
_season_boost  = bool(seasonality_boost)

# wine type
if _selected_type and _selected_type.lower() != "all":
    if 'full_type' in fallback_pool.columns:
        mask = fallback_pool['full_type'].astype(str).str.contains(_selected_type, case=False, na=False)
        fallback_pool = fallback_pool[mask]
    else:
        mask = False
        for c in [c for c in fallback_pool.columns if c.lower() in ('type','color','full_type')]:
            mask = mask | fallback_pool[c].astype(str).str.contains(_selected_type, case=False, na=False)
        fallback_pool = fallback_pool[mask]

# bottle size (ml)
if _selected_size:
    try:
        sel_ml = int(_selected_size)
        fallback_pool = fallback_pool[ fallback_pool['bottle_size_ml'].fillna(-1).astype(int) == sel_ml ]
    except Exception:
        pass

# last stock (<10)
if _last_stock:
    fallback_pool = fallback_pool[fallback_pool['stock'] < 10]

# seasonality boost (only if column exists and is truthy)
if _season_boost and 'seasonality_boost' in stock_df.columns:
    season_ids = set(
        stock_df.loc[
            stock_df['seasonality_boost'].apply(lambda x: bool(x) and str(x).strip() not in ('[]','False','false','0')),
            'id'
        ].astype(str)
    )
    fallback_pool = fallback_pool[fallback_pool['id'].astype(str).isin(season_ids)]

print(f"🧮 Fallback pool after filters: {len(fallback_pool)} rows")

# ---------- Rank (prefer higher CPI, then score, then stock) ----------
if 'avg_cpi' not in fallback_pool.columns:
    fallback_pool['avg_cpi'] = np.nan
fallback_pool['avg_cpi']   = pd.to_numeric(fallback_pool['avg_cpi'], errors='coerce')
fallback_pool['avg_score'] = pd.to_numeric(fallback_pool['avg_score'], errors='coerce')

fallback_pool = fallback_pool.sort_values(
    ['avg_cpi','avg_score','stock'], ascending=[False, False, False], na_position='last'
).reset_index(drop=True)

# Persist for the webapp/scheduler
fallback_path = OUTPUT_PATH / "fallback_pool.pkl"
fallback_pool.to_pickle(fallback_path)
print(f"💾 Saved fallback_pool → {fallback_path} (top {min(5, len(fallback_pool))} rows):")
display(fallback_pool.head(5))


🧮 Fallback pool after filters: 2389 rows
💾 Saved fallback_pool → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\fallback_pool.pkl (top 5 rows):


,id,wine,vintage,stock,avg_score,price_tier,full_type,region_group,bottle_size_ml,avg_cpi
0,53995,Trotanoy,2020,6,98.300000,Luxury,Still Red,Pomerol,750,0.561615
1,59622,Lodovico,2020,102,98.000000,Luxury,Still Red,Bolgheri,750,0.561615
2,51070,St. Eden,2016,3,98.000000,Ultra Luxury,Still Red,Napa Valley,750,0.561615
3,53882,Lafite Rothschild,2020,1716,97.923077,Ultra Luxury,Still Red,Pauillac,750,0.561615
4,62256,Cabernet Sauvignon Tychson Hill,2019,21,97.600000,Ultra Luxury,Still Red,Napa Valley,750,0.561615


In [23]:
# --- CELL 8 — Cadence Cards (Value Tue, Luxury Wed) [FIXED robust stock lookup] ---
import os, re, json
from datetime import datetime, timedelta
from pathlib import Path
import numpy as np
import pandas as pd

# ------------------------------
# 0) Paths & week
# ------------------------------
CALENDAR_PATH = OUTPUT_PATH / "calendar"
CALENDAR_PATH.mkdir(parents=True, exist_ok=True)

try:
    year = int(calendar_year)
    week = int(week_number)
except Exception:
    iso = datetime.now().isocalendar()
    year, week = iso.year, iso.week

week_root = CALENDAR_PATH / f"{year}_week_{week:02d}"
week_root.mkdir(parents=True, exist_ok=True)
latest_root = CALENDAR_PATH / "latest"
latest_root.mkdir(parents=True, exist_ok=True)

def _artifact_paths(name: str, ext: str = "json"):
    fname = f"{name}.{ext.lstrip('.')}"
    return (week_root / fname, latest_root / fname)

def _write_json(name: str, obj):
    wp, lp = _artifact_paths(name, "json")
    wp.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")
    lp.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")
    return wp, lp

def _tier(price):
    try:
        p = float(price)
    except Exception:
        return "Unknown"
    if p < 50:   return "Budget"
    if p < 100:  return "Mid-range"
    if p < 200:  return "Premium"
    if p < 500:  return "Luxury"
    return "Ultra Luxury"

# ------------------------------
# 1) Load inputs (single-read; reuse prior artifacts)
# ------------------------------
succ7_p  = OUTPUT_PATH / "campaign_success_7d.pkl"
succ14_p = OUTPUT_PATH / "campaign_success_14d.pkl"
succ30_p = OUTPUT_PATH / "campaign_success_30d.pkl"

succ7  = pd.read_pickle(succ7_p)  if succ7_p.exists()  else None
succ14 = pd.read_pickle(succ14_p) if succ14_p.exists() else None
succ30 = pd.read_pickle(succ30_p) if succ30_p.exists() else None

# Stock for enrichment
if 'stock_df' in globals() and isinstance(stock_df, pd.DataFrame):
    stock = stock_df.copy()
else:
    _sp = OUTPUT_PATH / "stock_df_final.pkl"
    stock = pd.read_pickle(_sp) if _sp.exists() else pd.DataFrame()
if not stock.empty and 'id' in stock.columns:
    stock['id'] = stock['id'].astype(str).str.strip()

# Clients / loyalty (derived from number of orders in Cell 6)
if 'client_pref_df' in globals() and isinstance(client_pref_df, pd.DataFrame):
    cpdf = client_pref_df.copy()
else:
    _cp = OUTPUT_PATH / "client_pref_df_latest.pkl"
    cpdf = pd.read_pickle(_cp) if _cp.exists() else pd.DataFrame()

group_counts = {}
if not cpdf.empty and 'loyalty_level' in cpdf.columns:
    group_counts = (cpdf['loyalty_level'].astype(str).str.lower()
                    .value_counts(dropna=False).to_dict())

# Power BI stats (for mapping campaign -> item list)
def _get_cached(name):
    try:
        return DATA_BUS.get("frames", {}).get(name)
    except Exception:
        return None

stats = _get_cached("powerbi_stats")
if stats is None:
    stats_path = SOURCE_PATH / "Power BI Dtld. Statistics ALL.xlsx"
    stats = pd.read_excel(stats_path) if stats_path.exists() else pd.DataFrame()

def _pick(df, *names, regex=None):
    if df is None or df.empty: return None
    cols = list(df.columns)
    if regex:
        m = re.compile(regex, re.I)
        for c in cols:
            if m.search(str(c)): return c
    for n in names:
        if n in cols: return n
    return None

col_camp = _pick(stats, "Campaign No.", "Campaign No", "Campaign Code", "CampaignNo", regex=r"\bcampaign\b")
col_item = _pick(stats, "Item No.", "Item No", "Item", "Item Number", regex=r"^item(\s|_)?(no|number)?$")
col_amt  = _pick(stats, "Total Bottle Amount (LCY)", regex=r"total.*bottle.*amount.*lcy")

if not stats.empty:
    stats['_Camp_No'] = stats[col_camp].astype(str).str.strip() if col_camp else ""
    stats['_Item_No'] = (stats[col_item].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)) if col_item else ""
    stats['_Amt']     = pd.to_numeric(stats[col_amt], errors="coerce").fillna(0.0) if col_amt else 0.0
else:
    stats = pd.DataFrame(columns=['_Camp_No','_Item_No','_Amt'])

# ------------------------------
# 2) Candidate winners from precomputed success pkl (or approximate fallback)
# ------------------------------
def _prep_succ(df):
    if df is None or df.empty:
        return pd.DataFrame(columns=["Campaign_No","Weighted_Score","Main_Sales","Conversion","Avg_Item_Price","Tier","campaign_date"])
    rename = {}
    for a, b in [("Campaign_No","Campaign_No"),("Campaign No","Campaign_No"),
                 ("Weighted_Score","Weighted_Score"),("Main_Sales","Main_Sales"),
                 ("Conversion","Conversion"),("Avg_Item_Price","Avg_Item_Price"),
                 ("campaign_date","campaign_date")]:
        if a in df.columns: rename[a] = b
    d = df.rename(columns=rename).copy()
    if "Avg_Item_Price" not in d.columns: d["Avg_Item_Price"] = np.nan
    d["Tier"] = d["Avg_Item_Price"].apply(_tier)
    return d[["Campaign_No","Weighted_Score","Main_Sales","Conversion","Avg_Item_Price","Tier","campaign_date"]]

cand = pd.concat([_prep_succ(succ30), _prep_succ(succ14), _prep_succ(succ7)], ignore_index=True)
cand = cand.dropna(subset=["Campaign_No"]).copy()

if cand.empty:
    # Fallback: approximate score from sales only (no re-reading OMT)
    if stats.empty or (stats['_Camp_No'] == "").all():
        cand = pd.DataFrame(columns=["Campaign_No","Weighted_Score","Main_Sales","Conversion","Avg_Item_Price","Tier","campaign_date"])
    else:
        grp = stats.groupby('_Camp_No', as_index=False).agg(Main_Sales=('_Amt','sum'))
        grp["Conversion"] = np.nan
        # try to find a price column as proxy
        col_price = _pick(stats, "Item Sales Price from Price List", "Item Sales Price", regex=r"(item\s*)?sales\s*price.*list")
        if col_price:
            price_df = stats.groupby('_Camp_No', as_index=False)[col_price].median().rename(columns={col_price:"Avg_Item_Price"})
        else:
            price_df = pd.DataFrame({'_Camp_No': grp['_Camp_No'], 'Avg_Item_Price': np.nan})
        cand = grp.merge(price_df, on='_Camp_No', how='left').rename(columns={'_Camp_No':'Campaign_No'})
        m = float(cand["Main_Sales"].max() or 0.0)
        cand["Weighted_Score"] = (cand["Main_Sales"] / m) if m > 0 else 0.0
        cand["campaign_date"] = pd.NaT
        cand["Tier"] = cand["Avg_Item_Price"].apply(_tier)

if not cand.empty:
    cand = (cand.sort_values(["Weighted_Score","Main_Sales"], ascending=[False, False])
                 .drop_duplicates(["Campaign_No"], keep="first"))

# ------------------------------
# 3) Pick Value (Tue) & Luxury (Wed)
# ------------------------------
def _pick_by_tiers(df, tiers):
    if df is None or df.empty: return None
    sub = df[df["Tier"].isin(tiers)].copy()
    if sub.empty: return None
    sub = sub.sort_values(["Weighted_Score","Main_Sales","Conversion"], ascending=[False, False, False])
    return sub.iloc[0].to_dict()

value_tiers = {"Budget","Mid-range","Premium"}
lux_tiers   = {"Luxury","Ultra Luxury"}

value_win = _pick_by_tiers(cand, value_tiers)
lux_win   = _pick_by_tiers(cand, lux_tiers)

# ------------------------------
# 4) Map campaign → one or more item candidates (sorted by sales)
# ------------------------------
def _items_for_campaign(camp_no: str) -> list[str]:
    if not camp_no or stats.empty: return []
    sub = stats.loc[stats['_Camp_No'].astype(str) == str(camp_no), ['_Item_No','_Amt']].copy()
    sub = sub[sub['_Item_No'].astype(str).str.strip() != ""]
    if sub.empty: return []
    sub = sub.groupby('_Item_No', as_index=False).agg(Main_Sales=('_Amt','sum'))
    sub = sub.sort_values("Main_Sales", ascending=False)
    return [str(x) for x in sub['_Item_No'].tolist()]

value_items = _items_for_campaign(value_win["Campaign_No"]) if value_win else []
lux_items   = _items_for_campaign(lux_win["Campaign_No"])   if lux_win   else []

# ------------------------------
# 5) Safe stock meta lookup + card builder (robust to missing stock rows)
# ------------------------------
def _get_stock_meta(item_id: str) -> dict:
    if not item_id or stock.empty or 'id' not in stock.columns: 
        return {}
    try:
        recs = stock.loc[stock['id'].astype(str) == str(item_id)].head(1).to_dict("records")
        return recs[0] if recs else {}
    except Exception:
        return {}

def _mk_card_from_candidates(item_ids: list[str], tier_label: str, camp_row: dict, day: str, tag: str):
    # try each item until one exists in stock; else fall back to the first id with minimal meta
    chosen_id = None
    meta = {}
    for iid in item_ids:
        meta = _get_stock_meta(iid)
        if meta:
            chosen_id = iid
            break
    if chosen_id is None and item_ids:
        chosen_id = item_ids[0]  # minimal fallback (might not be in stock dataset)
        meta = {}

    if not chosen_id:
        return None  # nothing to emit

    label = meta.get("wine") or meta.get("Wine") or f"Item {chosen_id}"
    vint  = str(meta.get("vintage","NV") or "NV")
    price_tier = str(meta.get("price_tier") or tier_label or "")
    region_group = str(meta.get("region_group") or meta.get("region") or "")
    full_type    = str(meta.get("full_type") or "")
    chf_price    = (float(meta["CHF Price"]) if ("CHF Price" in meta and pd.notna(meta["CHF Price"])) else None)
    avg_cpi      = (float(meta["avg_cpi_score"]) if ("avg_cpi_score" in meta and pd.notna(meta["avg_cpi_score"])) else 0.0)
    avg_score    = (float(meta["avg_score"]) if ("avg_score" in meta and pd.notna(meta["avg_score"])) else None)
    stock_qty    = int(pd.to_numeric(meta.get("stock", 0), errors="coerce") or 0)

    card = {
        "id": str(chosen_id),
        "wine": label,
        "vintage": vint,
        "full_type": full_type or "Unknown",
        "region_group": region_group or "Unknown",
        "price_tier": price_tier,
        "stock": stock_qty,
        "match_quality": f"Cadence ({tag})",
        "avg_cpi_score": avg_cpi,
        "locked": False,
        "extras": {
            "producer": meta.get("producer"),
            "chf_price": chf_price,
            "avg_score": avg_score,
        },
        "source_campaign_no": camp_row.get("Campaign_No") if camp_row else None,
        "source_weighted_score": float(camp_row.get("Weighted_Score", 0.0)) if camp_row else 0.0,
        "source_main_sales": float(camp_row.get("Main_Sales", 0.0)) if camp_row else 0.0,
        "source_tier": tier_label,
        "scheduled_day_hint": day,
    }
    if camp_row and camp_row.get("campaign_date"):
        try:
            card["last_campaign_date"] = pd.to_datetime(camp_row["campaign_date"]).date().isoformat()
        except Exception:
            pass
    return card

tue_card = _mk_card_from_candidates(value_items, value_win.get("Tier") if value_win else None, value_win, "Tuesday", "Value Winner") if value_win else None
wed_card = _mk_card_from_candidates(lux_items,   lux_win.get("Tier")   if lux_win   else None, lux_win,   "Wednesday", "Luxury Winner") if lux_win else None

# ------------------------------
# 6) Persist cadence cards (+ meta)
# ------------------------------
cadence_payload = {
    "week": {"year": year, "week_number": week},
    "cards": {
        "Tuesday":  [tue_card] if tue_card else [],
        "Wednesday":[wed_card] if wed_card else [],
    },
    "meta": {
        "computed_at": datetime.now().isoformat(timespec="seconds"),
        "group_counts": group_counts,     # Bronze/Silver/Gold/VIP distribution
        "sources": {
            "campaign_success_7d":  str(succ7_p)  if succ7_p.exists() else None,
            "campaign_success_14d": str(succ14_p) if succ14_p.exists() else None,
            "campaign_success_30d": str(succ30_p) if succ30_p.exists() else None,
            "stats_file":           str(SOURCE_PATH / "Power BI Dtld. Statistics ALL.xlsx"),
            "stock_pkl":            str(OUTPUT_PATH / "stock_df_final.pkl"),
        }
    }
}

_wp, _lp = _write_json("cadence_cards", cadence_payload)
print(f"✅ Cadence cards saved → { _wp } and alias { _lp }")
if tue_card: print("• Tue (Value):", tue_card["wine"], "→", tue_card["source_campaign_no"], "| item:", tue_card["id"])
if wed_card: print("• Wed (Luxury):", wed_card["wine"], "→", wed_card["source_campaign_no"], "| item:", wed_card["id"])


✅ Cadence cards saved → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\calendar\2025_week_38\cadence_cards.json and alias C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\calendar\latest\cadence_cards.json
• Tue (Value): Item 64103 → CM-25-02208 | item: 64103
• Wed (Luxury): Angélus Hommage à Elisabeth Bouchet → CM-25-02273 | item: 63044


In [24]:
# --- CELL 9 — Resolve winners → stock ids, build winner cards (no re-imports) ---
import json, re
from datetime import datetime, timedelta
from pathlib import Path
import numpy as np
import pandas as pd

# ------------------------------
# 0) Paths & week context
# ------------------------------
try:
    year = int(calendar_year)
    week = int(week_number)
except Exception:
    _iso = datetime.now().isocalendar()
    year, week = _iso.year, _iso.week

CALENDAR_PATH = OUTPUT_PATH / "calendar"
WEEK_ROOT     = CALENDAR_PATH / f"{year}_week_{week:02d}"
LATEST_ROOT   = CALENDAR_PATH / "latest"
for p in (CALENDAR_PATH, WEEK_ROOT, LATEST_ROOT): p.mkdir(parents=True, exist_ok=True)

def _artifact_paths(name: str, ext: str = "json"):
    fname = f"{name}.{ext.lstrip('.')}"
    return (WEEK_ROOT / fname, LATEST_ROOT / fname)

def _write_json(name: str, obj):
    wp, lp = _artifact_paths(name, "json")
    txt = json.dumps(obj, ensure_ascii=False, indent=2)
    wp.write_text(txt, encoding="utf-8")
    lp.write_text(txt, encoding="utf-8")
    return wp, lp

# ------------------------------
# 1) Reuse cached frames / single-read policy
# ------------------------------
def _get_cached(name):
    try:
        return DATA_BUS.get("frames", {}).get(name)
    except Exception:
        return None

# Stock snapshot (prefer in-memory)
if 'stock_df' in globals() and isinstance(stock_df, pd.DataFrame):
    _stock = stock_df.copy()
else:
    _sp = OUTPUT_PATH / "stock_df_final.pkl"
    _stock = pd.read_pickle(_sp) if _sp.exists() else pd.DataFrame()
if not _stock.empty and 'id' in _stock.columns:
    _stock['id'] = _stock['id'].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)

# Client prefs (for meta group counts)
if 'client_pref_df' in globals() and isinstance(client_pref_df, pd.DataFrame):
    _cpdf = client_pref_df.copy()
else:
    _cp = OUTPUT_PATH / "client_pref_df_latest.pkl"
    _cpdf = pd.read_pickle(_cp) if _cp.exists() else pd.DataFrame()
group_counts = {}
if not _cpdf.empty and 'loyalty_level' in _cpdf.columns:
    group_counts = (_cpdf['loyalty_level'].astype(str).str.lower()
                    .value_counts(dropna=False).to_dict())

# Power BI stats for mapping Campaign -> Item list (prefer cache)
_stats = _get_cached("powerbi_stats")
if _stats is None:
    _stats_path = SOURCE_PATH / "Power BI Dtld. Statistics ALL.xlsx"
    _stats = pd.read_excel(_stats_path) if _stats_path.exists() else pd.DataFrame()

def _pick(df, *names, regex=None):
    if df is None or df.empty: return None
    cols = list(df.columns)
    if regex:
        rx = re.compile(regex, re.I)
        for c in cols:
            if rx.search(str(c)): return c
    for n in names:
        if n in cols: return n
    return None

_col_camp = _pick(_stats, "Campaign No.", "Campaign Code", "CampaignNo", regex=r"\bcampaign\b")
_col_item = _pick(_stats, "Item No.", "Item", "Item Number", regex=r"^item(\s|_)?(no|number)?$")
_col_amt  = _pick(_stats, "Total Bottle Amount (LCY)", regex=r"total.*bottle.*amount.*lcy")
_stats = _stats.copy()
if not _stats.empty:
    _stats["__camp"]   = _stats.get(_col_camp, pd.Series("", index=_stats.index)).astype(str).str.strip()
    _stats["__itemno"] = _stats.get(_col_item, pd.Series("", index=_stats.index)).astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
    _stats["__amt"]    = pd.to_numeric(_stats.get(_col_amt, 0), errors="coerce").fillna(0.0)
else:
    _stats = pd.DataFrame(columns=["__camp","__itemno","__amt"])

# ------------------------------
# 2) Load precomputed winners (7/14/30d) from Cell 8’s pipeline
# ------------------------------
def _load_best(days: int):
    p = OUTPUT_PATH / f"campaign_success_{days}d.pkl"
    if not p.exists() or p.stat().st_size == 0: return None
    df = pd.read_pickle(p)
    if df is None or df.empty: return None
    # normalize expected columns
    ren = {}
    for a,b in [("Campaign_No","Campaign_No"),("Campaign No","Campaign_No"),
                ("Weighted_Score","Weighted_Score"),("Main_Sales","Main_Sales"),
                ("Conversion","Conversion"),("Avg_Item_Price","Avg_Item_Price"),("campaign_date","campaign_date")]:
        if a in df.columns: ren[a] = b
    d = df.rename(columns=ren).copy()
    d = d.sort_values(["Weighted_Score","Main_Sales","Conversion"], ascending=[False, False, False])
    return d.iloc[0].to_dict()

best7  = _load_best(7)
best14 = _load_best(14)
best30 = _load_best(30)

# ------------------------------
# 3) Map each winning campaign → candidate items (sorted by sales)
# ------------------------------
def _items_for_campaign(camp_no: str) -> list[str]:
    if not camp_no or _stats.empty: return []
    sub = _stats.loc[_stats["__camp"].astype(str) == str(camp_no), ["__itemno","__amt"]].copy()
    sub = sub[sub["__itemno"].astype(str).str.strip() != ""]
    if sub.empty: return []
    g = (sub.groupby("__itemno", as_index=False)["__amt"].sum()
             .sort_values("__amt", ascending=False))
    return [str(x) for x in g["__itemno"].tolist()]

cand7  = _items_for_campaign(best7.get("Campaign_No"))  if best7  else []
cand14 = _items_for_campaign(best14.get("Campaign_No")) if best14 else []
cand30 = _items_for_campaign(best30.get("Campaign_No")) if best30 else []

# ------------------------------
# 4) Resolve to a stock-backed card (robust to missing stock rows)
# ------------------------------
def _tier(price):
    try:
        p = float(price)
    except Exception:
        return ""
    if p < 50: return "Budget"
    if p < 100: return "Mid-range"
    if p < 200: return "Premium"
    if p < 500: return "Luxury"
    return "Ultra Luxury"

def _choose_stock(item_ids: list[str]) -> tuple[str|None, dict]:
    if not item_ids: return None, {}
    for iid in item_ids:
        if _stock.empty or 'id' not in _stock.columns: break
        recs = _stock.loc[_stock['id'].astype(str) == str(iid)].head(1).to_dict("records")
        if recs: return str(iid), recs[0]
    # fallback to first id without meta
    return (str(item_ids[0]), {}) if item_ids else (None, {})

def _mk_winner_card(tag: str, best_row: dict, item_ids: list[str]):
    if not best_row: return None
    chosen_id, meta = _choose_stock(item_ids)
    if not chosen_id: return None
    tier_name = _tier(best_row.get("Avg_Item_Price"))
    card = {
        "id": chosen_id,
        "wine": meta.get("wine") or f"Campaign {best_row.get('Campaign_No')}",
        "name": meta.get("wine") or f"Campaign {best_row.get('Campaign_No')}",
        "vintage": str(meta.get("vintage") or "NV"),
        "full_type": meta.get("full_type") or "Unknown",
        "region_group": meta.get("region_group") or "Unknown",
        "stock": int(pd.to_numeric(meta.get("stock", 0), errors="coerce") or 0),
        "price_tier": meta.get("price_tier") or tier_name or "",
        "match_quality": f"Winner {tag}",
        "avg_cpi_score": float(meta.get("avg_cpi_score")) if pd.notna(meta.get("avg_cpi_score", np.nan)) else 0.0,
        "locked": True,
        "reason": f"Top campaign ({tag}) — score {round(float(best_row.get('Weighted_Score',0)),3)}; "
                  f"conv {round(float(best_row.get('Conversion',0))*100,2)}%; "
                  f"sales {round(float(best_row.get('Main_Sales',0)),2)}",
        "source_campaign_no": best_row.get("Campaign_No"),
    }
    if best_row.get("campaign_date"):
        try:
            card["last_campaign_date"] = pd.to_datetime(best_row["campaign_date"]).date().isoformat()
        except Exception:
            pass
    return card

# Place winners Mon/Thu/Fri (leave Wed free for Luxury cadence card from Cell 8)
winners_cards = {
    "Monday":   [_mk_winner_card("7d",  best7,  cand7)]  if best7  else [],
    "Thursday": [_mk_winner_card("14d", best14, cand14)] if best14 else [],
    "Friday":   [_mk_winner_card("30d", best30, cand30)] if best30 else [],
}
# prune None
for d in list(winners_cards.keys()):
    winners_cards[d] = [x for x in winners_cards[d] if x]

# ------------------------------
# 5) Persist winners (calendar week + latest + legacy alias)
# ------------------------------
payload = {
    "week": {"year": year, "week_number": week},
    "cards": winners_cards,
    "meta": {
        "computed_at": datetime.now().isoformat(timespec="seconds"),
        "group_counts": group_counts,
        "sources": {
            "campaign_success_7d":  str((OUTPUT_PATH / "campaign_success_7d.pkl")),
            "campaign_success_14d": str((OUTPUT_PATH / "campaign_success_14d.pkl")),
            "campaign_success_30d": str((OUTPUT_PATH / "campaign_success_30d.pkl")),
            "stats_file":           str(SOURCE_PATH / "Power BI Dtld. Statistics ALL.xlsx"),
            "stock_pkl":            str(OUTPUT_PATH / "stock_df_final.pkl"),
        }
    }
}
wp, lp = _write_json("winner_cards", payload)
print(f"🏆 Winner cards saved → {wp} and alias {lp}")

# Legacy flat file (for older injector code, if any)
legacy_path = OUTPUT_PATH / "winner_cards_current_week.json"
legacy_flat = {day: cards for day, cards in winners_cards.items()}
legacy_path.write_text(json.dumps(legacy_flat, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"↩️  Legacy winners alias written → {legacy_path}")

# NOTE: We do NOT inject into a 'locked_json' here; Cell 11 should merge
#       both 'winner_cards.json' and 'cadence_cards.json' into the calendar.


🏆 Winner cards saved → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\calendar\2025_week_38\winner_cards.json and alias C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\calendar\latest\winner_cards.json
↩️  Legacy winners alias written → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\winner_cards_current_week.json


In [26]:
# --- CELL 10 — Cadence winners (Tue/Wed) → attach frequency groups (order-count based) ---
import json, re
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd

# ------------------------------
# 0) Week + paths (reuse from Cell 1)
# ------------------------------
try:
    year = int(calendar_year)
    week = int(week_number)
except Exception:
    _iso = datetime.now().isocalendar()
    year, week = _iso.year, _iso.week

CALENDAR_PATH = OUTPUT_PATH / "calendar"
WEEK_ROOT     = CALENDAR_PATH / f"{year}_week_{week:02d}"
LATEST_ROOT   = CALENDAR_PATH / "latest"
for p in (CALENDAR_PATH, WEEK_ROOT, LATEST_ROOT): p.mkdir(parents=True, exist_ok=True)

CADENCE_WEEK   = WEEK_ROOT / "cadence_cards.json"
CADENCE_LATEST = LATEST_ROOT / "cadence_cards.json"
LEGACY_ALIAS   = OUTPUT_PATH / "cadence_cards_current_week.json"

def _write_jsons(obj: dict):
    txt = json.dumps(obj, ensure_ascii=False, indent=2)
    CADENCE_WEEK.write_text(txt, encoding="utf-8")
    CADENCE_LATEST.write_text(txt, encoding="utf-8")

# ------------------------------
# 1) Load cadence cards from Cell 8 (required)
# ------------------------------
if CADENCE_WEEK.exists() and CADENCE_WEEK.stat().st_size > 0:
    cadence_payload = json.loads(CADENCE_WEEK.read_text(encoding="utf-8"))
elif CADENCE_LATEST.exists() and CADENCE_LATEST.stat().st_size > 0:
    cadence_payload = json.loads(CADENCE_LATEST.read_text(encoding="utf-8"))
else:
    # Minimal fallback (empty skeleton); later cells can still run
    cadence_payload = {"week":{"year":year,"week_number":week},"cards":{"Tuesday":[],"Wednesday":[]},"meta":{}}

cards = cadence_payload.setdefault("cards", {})
tue_card = (cards.get("Tuesday") or [None])[0]
wed_card = (cards.get("Wednesday") or [None])[0]

# ------------------------------
# 2) Compute client frequency groups from order counts
#     (reuse cached stats if present; else single read)
# ------------------------------
def _get_cached(name):
    try:
        return DATA_BUS.get("frames", {}).get(name)
    except Exception:
        return None

_stats = _get_cached("powerbi_stats")
if _stats is None:
    stats_path = SOURCE_PATH / "Power BI Dtld. Statistics ALL.xlsx"
    _stats = pd.read_excel(stats_path) if stats_path.exists() else pd.DataFrame()

def _pick(df, *names, regex=None):
    if df is None or df.empty: return None
    cols = list(df.columns)
    if regex:
        rx = re.compile(regex, re.I)
        for c in cols:
            if rx.search(str(c)): return c
    for n in names:
        if n in cols: return n
    return None

col_cust = _pick(_stats, "Customer No.", "Customer No", "CustomerNumber", regex=r"customer.*no")
col_doc  = _pick(_stats, "Document No.", "Document No", "Order No.", "Order No", regex=r"(document|order).*(no|number|id)")
col_date = _pick(_stats, "Posting Date", "Document Date", "Sales Date", "Date", regex=r"(posting|document|sales).*(date)")

if not _stats.empty and col_cust:
    stats2 = _stats[[col_cust]].copy()
    stats2[col_cust] = stats2[col_cust].astype(str).str.strip().str.replace(r"\.0$","", regex=True)
    if col_doc:
        # Count DISTINCT orders per customer (preferred)
        tmp = _stats[[col_cust, col_doc]].copy()
        tmp[col_doc] = tmp[col_doc].astype(str).str.strip()
        order_counts = (tmp.dropna()
                          .drop_duplicates([col_cust, col_doc])
                          .groupby(col_cust).size()
                          .rename("order_count")
                          .reset_index())
    else:
        # Fallback: count rows per customer
        order_counts = (stats2.groupby(col_cust).size()
                             .rename("order_count")
                             .reset_index())
else:
    order_counts = pd.DataFrame(columns=[col_cust or "customer_no","order_count"])

# Map order_count → cadence group
def _cadence_group(n: float|int) -> str:
    try:
        n = int(n)
    except Exception:
        return "month"  # safest, least frequent
    if n >= 20: return "tris"     # 3x/week candidates
    if n >= 10: return "bis"      # 2x/week (alias)
    if n >= 5:  return "twice"    # 2x/week (canonical)
    if n >= 2:  return "one"      # 1x/week
    return "month"                # monthly

if not order_counts.empty:
    order_counts["cadence_group"] = order_counts["order_count"].apply(_cadence_group)
    group_counts = order_counts["cadence_group"].value_counts(dropna=False).to_dict()
else:
    group_counts = {}

# Expose in meta (along with mapping rules)
cadence_payload.setdefault("meta", {})
cadence_payload["meta"]["group_counts"] = group_counts
cadence_payload["meta"]["group_rules"] = {
    "tris":  "≥20 orders",
    "bis":   "10–19 orders (2x/week alias)",
    "twice": "5–9 orders (2x/week)",
    "one":   "2–4 orders (1x/week)",
    "month": "0–1 orders (monthly)"
}

# ------------------------------
# 3) Assign cadence_groups to Tue/Wed cards based on bucket
# ------------------------------
def _bucket_from_price_tier(tier: str|None, price_hint=None) -> str:
    t = (tier or "").strip().lower()
    if t in {"budget","mid-range","mid range","premium"}: return "value"
    if t in {"luxury","ultra luxury","ultraluxury"}: return "lux"
    if price_hint is not None:
        try:
            p = float(price_hint);  return "value" if p < 200 else "lux"
        except Exception:
            pass
    return "value"

def _recommended_groups(bucket: str, counts: dict) -> list[str]:
    pref = ["tris","bis","twice"] if bucket == "lux" else ["month","one","twice"]
    out = [g for g in pref if counts.get(g, 0) > 0]
    return out or pref[:2]

def _attach_groups(card: dict|None) -> dict|None:
    if not card: return None
    card = dict(card)  # copy
    # Backfill missing fields to avoid KeyErrors downstream
    card["name"] = card.get("name") or card.get("wine") or f"Item {card.get('id','')}".strip()
    card["price_tier"] = card.get("price_tier") or ""
    bucket = _bucket_from_price_tier(card.get("price_tier"))
    card["campaign_tag"] = "cadence"
    card["locked"] = True
    card["cadence_groups"] = _recommended_groups(bucket, group_counts)
    return card

if tue_card:
    tue_card = _attach_groups(tue_card)
if wed_card:
    wed_card = _attach_groups(wed_card)

# Put them back (keep structure stable even if missing)
cadence_payload["cards"]["Tuesday"]   = [tue_card] if tue_card else []
cadence_payload["cards"]["Wednesday"] = [wed_card] if wed_card else []

# ------------------------------
# 4) Persist (week + latest + legacy alias)
# ------------------------------
_write_jsons(cadence_payload)

legacy_flat = {"Tuesday": cadence_payload["cards"]["Tuesday"],
               "Wednesday": cadence_payload["cards"]["Wednesday"]}
LEGACY_ALIAS.write_text(json.dumps(legacy_flat, ensure_ascii=False, indent=2), encoding="utf-8")

print(f"📦 Cadence cards updated with groups → {CADENCE_WEEK}")
for d in ("Tuesday","Wednesday"):
    arr = cadence_payload["cards"].get(d, [])
    if arr:
        c = arr[0]
        nm = c.get("name") or c.get("wine") or str(c.get("id",""))
        pt = c.get("price_tier","")
        gr = c.get("cadence_groups", [])
        print(f"• {d}: {nm} | tier={pt} | groups={gr}")
    else:
        print(f"• {d}: (none)")
print(f"↩️  Legacy alias written → {LEGACY_ALIAS}")


📦 Cadence cards updated with groups → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\calendar\2025_week_38\cadence_cards.json
• Tuesday: Item 64103 | tier=Mid-range | groups=['month', 'one', 'twice']
• Wednesday: Angélus Hommage à Elisabeth Bouchet | tier=Ultra Luxury | groups=['tris', 'bis', 'twice']
↩️  Legacy alias written → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\cadence_cards_current_week.json


In [28]:
# --- CELL 11 — Compose weekly calendar; add special Wednesday overflow clones ---
import json
from datetime import datetime
from pathlib import Path

# Week + paths
try:
    year = int(calendar_year)
    week = int(week_number)
except Exception:
    iso = datetime.now().isocalendar()
    year, week = iso.year, iso.week

CALENDAR_PATH = OUTPUT_PATH / "calendar"
WEEK_ROOT     = CALENDAR_PATH / f"{year}_week_{week:02d}"
LATEST_ROOT   = CALENDAR_PATH / "latest"
LOCKED_PATH   = OUTPUT_PATH / "locked_weeks"
for p in (CALENDAR_PATH, WEEK_ROOT, LATEST_ROOT, LOCKED_PATH): p.mkdir(parents=True, exist_ok=True)

NUM_SLOTS_SAFE = int(globals().get("NUM_SLOTS", 5))
DAYS_FULL_SAFE = list(globals().get("DAYS_FULL", ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]))

def _load_json(path: Path):
    try:
        if path.exists() and path.stat().st_size > 0:
            return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

# Load parts produced by Cells 8–10
winner_week   = _load_json(WEEK_ROOT / "winner_cards.json")   or _load_json(LATEST_ROOT / "winner_cards.json")   or {}
cadence_week  = _load_json(WEEK_ROOT / "cadence_cards.json")  or _load_json(LATEST_ROOT / "cadence_cards.json")  or {}

w_cards = (winner_week.get("cards") or {}) if isinstance(winner_week, dict) else {}
c_cards = (cadence_week.get("cards") or {}) if isinstance(cadence_week, dict) else {}

# Merge order: winners first (usually locked), then cadence (also locked)
merged = {d: [] for d in DAYS_FULL_SAFE}

def _sig_val(x): 
    return "" if x is None else str(x).strip()

def _signature(card):
    if not isinstance(card, dict): return None
    return (
        _sig_val(card.get("id")),
        _sig_val(card.get("name") or card.get("wine")),
        _sig_val(card.get("source_campaign_no") or card.get("campaign_no")),
        _sig_val(card.get("campaign_tag")),
        _sig_val(card.get("audience_group")),    # ← include group so clones are distinct
        _sig_val(card.get("clone_key")),         # ← extra safeguard
    )

def _dedup_append(target_list, items):
    seen = { _signature(x) for x in target_list if isinstance(x, dict) }
    for it in (items or []):
        sig = _signature(it)
        if sig and sig not in seen:
            target_list.append(it); seen.add(sig)

for day in DAYS_FULL_SAFE:
    _dedup_append(merged[day], w_cards.get(day, []))
    _dedup_append(merged[day], c_cards.get(day, []))

# --- Special Wednesday overflow: clone the Luxury cadence card per audience group ---
SPECIAL_WEDNESDAY_OVERFLOW = True

def _is_luxury_card(card: dict) -> bool:
    t = _sig_val(card.get("price_tier")).lower()
    return t in {"luxury","ultra luxury"}

def _clone_for_groups(card: dict, groups: list[str]) -> list[dict]:
    clones = []
    base_id = _sig_val(card.get("id"))
    base_name = card.get("name") or card.get("wine") or f"Item {base_id}"
    for g in (groups or []):
        clone = dict(card)
        clone["audience_group"] = g
        clone["match_quality"] = (card.get("match_quality") or "cadence") + f" | group:{g}"
        clone["clone_key"] = f"{base_id}::WED::{g}"
        clone["locked"] = True
        # keep same id/wine/etc., only audience changes
        clones.append(clone)
    return clones

if SPECIAL_WEDNESDAY_OVERFLOW:
    wed_items = merged.get("Wednesday", [])
    # find the luxury cadence card (we expect just one from Cell 10)
    lux_card = None
    for it in wed_items:
        if isinstance(it, dict) and _is_luxury_card(it):
            lux_card = it
            break
    if lux_card:
        groups = lux_card.get("cadence_groups") or []   # e.g., ['tris','bis','twice']
        # create one overflow clone per audience group
        clones = _clone_for_groups(lux_card, groups)
        # ensure they don't end up in main; we'll append later (after main fill)
        # stash to apply after main/overflow split
    else:
        clones = []
else:
    clones = []

# Build main/overflow respecting NUM_SLOTS
calendar_out = {d: {"main": [], "overflow": []} for d in DAYS_FULL_SAFE}

for day in DAYS_FULL_SAFE:
    items = merged.get(day, [])
    main = items[:NUM_SLOTS_SAFE]
    overflow = items[NUM_SLOTS_SAFE:]
    # Inject special Wednesday clones into overflow only
    if day == "Wednesday" and clones:
        # avoid duplicating the exact same object if already present
        _dedup_append(overflow, clones)
    calendar_out[day]["main"] = main
    calendar_out[day]["overflow"] = overflow

# Persist composed week + alias + a compact “weekly_calendar” view for consumers
compose_payload = {
    "week": {"year": year, "week_number": week},
    "updated_at": datetime.now().isoformat(timespec="seconds"),
    "weekly_calendar_struct": calendar_out,
    "weekly_calendar": { d: calendar_out[d]["main"] for d in DAYS_FULL_SAFE },
    "sources": {
        "winner_cards":  (WEEK_ROOT / "winner_cards.json").name if (WEEK_ROOT / "winner_cards.json").exists() else None,
        "cadence_cards": (WEEK_ROOT / "cadence_cards.json").name if (WEEK_ROOT / "cadence_cards.json").exists() else None,
        "num_slots": NUM_SLOTS_SAFE,
    }
}

def _write(name, obj):
    txt = json.dumps(obj, ensure_ascii=False, indent=2)
    (WEEK_ROOT / f"{name}.json").write_text(txt, encoding="utf-8")
    (LATEST_ROOT / f"{name}.json").write_text(txt, encoding="utf-8")

_write("calendar_composed", compose_payload)

# Locked snapshot stays main-only (UI expects fixed NUM_SLOTS)
locked_week = { day: [ (calendar_out[day]["main"][i] if i < len(calendar_out[day]["main"]) else None)
                       for i in range(NUM_SLOTS_SAFE) ]
               for day in DAYS_FULL_SAFE }
locked_path = LOCKED_PATH / f"locked_calendar_{year}_week_{week}.json"
locked_path.write_text(json.dumps(locked_week, ensure_ascii=False, indent=2), encoding="utf-8")

print("🧩 Calendar composed:")
for d in DAYS_FULL_SAFE:
    m = len(calendar_out[d]["main"]); o = len(calendar_out[d]["overflow"])
    print(f"  • {d}: {m} main, {o} overflow")

print(f"✅ Wrote composed → {WEEK_ROOT / 'calendar_composed.json'} and alias {LATEST_ROOT / 'calendar_composed.json'}")
print(f"🔒 Locked snapshot → {locked_path}")


🧩 Calendar composed:
  • Monday: 1 main, 0 overflow
  • Tuesday: 1 main, 0 overflow
  • Wednesday: 1 main, 3 overflow
  • Thursday: 1 main, 0 overflow
  • Friday: 1 main, 0 overflow
  • Saturday: 0 main, 0 overflow
  • Sunday: 0 main, 0 overflow
✅ Wrote composed → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\calendar\2025_week_38\calendar_composed.json and alias C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\calendar\latest\calendar_composed.json
🔒 Locked snapshot → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\locked_weeks\locked_calendar_2025_week_38.json


In [29]:
# ---  CELL 12: Save UI-Ready JSON & PKL for Flask ---

import os
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from time import time
start_time = time()

# ---------- Fallbacks if earlier cells didn't run ----------
try:
    OUTPUT_PATH
except NameError:
    _default_output = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
    OUTPUT_PATH = Path(os.getenv("OUTPUT_PATH", _default_output))
    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

try:
    NUM_SLOTS
except NameError:
    NUM_SLOTS = 5

# Load stock if not in memory
if 'stock_df' not in globals():
    _stock_pkl = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df = pd.read_pickle(_stock_pkl) if _stock_pkl.exists() else pd.DataFrame()

# Load client pref if not in memory
if 'client_pref_df' not in globals():
    _pref_pkl = OUTPUT_PATH / "client_pref_df_latest.pkl"
    client_pref_df = pd.read_pickle(_pref_pkl) if _pref_pkl.exists() else pd.DataFrame(columns=['customer_no'])

# ---------- Locked snapshot helpers ----------
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

def _load_json(path: Path):
    try:
        if path.exists() and path.stat().st_size > 0:
            return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

def _locked_to_df(locked_json: dict, stock_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for day in DAYS:
        for it in (locked_json.get(day) or []):
            if not it: 
                continue
            rows.append({"day": day, **it})
    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df["id"] = df.get("id","").astype(str)
    enrich_cols = ["id","wine","vintage","full_type","region_group","stock","price_tier","avg_cpi_score"]
    have = [c for c in enrich_cols if c in stock_df.columns]
    if have:
        df = df.merge(stock_df[have].drop_duplicates("id"), on="id", how="left")
    df["locked"] = True
    # Title-case price_tier if present; otherwise create an empty column
    if "price_tier" in df.columns:
        df["price_tier"] = df["price_tier"].apply(
            lambda x: x.title().strip() if isinstance(x, str) and x else ""
        )
    else:
        df["price_tier"] = ""
    return df


# ---------- Determine selected year/week and load locked snapshot ----------
try:
    sel_year = int(calendar_year)
except Exception:
    sel_year = datetime.now().year
try:
    sel_week = int(week_number)
except Exception:
    sel_week = datetime.now().isocalendar().week

LOCKED_PATH = OUTPUT_PATH / "locked_weeks"
LOCKED_PATH.mkdir(parents=True, exist_ok=True)

persisted = _load_json(LOCKED_PATH / f"locked_calendar_{sel_year}_week_{sel_week}.json")

# Prefer in-memory structured snapshot from Cell 1; else persisted; else empty
if "locked_calendar_snapshot" in globals() and locked_calendar_snapshot:
    # Convert structured snapshot {day:{main,overflow}} → flat lists per day
    locked_json = {d: (locked_calendar_snapshot.get(d, {}).get("main") or []) for d in DAYS}
elif persisted:
    locked_json = persisted
else:
    locked_json = {d: [] for d in DAYS}

# Now that we have locked_json and stock_df, build locked_df
locked_df = _locked_to_df(locked_json, stock_df)

# --- Load winner boxes built in Cell 15 (optional) ---
WINNERS_PATH = OUTPUT_PATH / "winners_boxes.json"
try:
    winner_boxes = json.loads(WINNERS_PATH.read_text(encoding="utf-8")) if WINNERS_PATH.exists() else {}
except Exception as _e:
    print(f"⚠️ Could not read winners_boxes.json: {_e}")
    winner_boxes = {}


# ---------- Tier helpers ----------
TIER_MAP_NAME = {
    "budget": "Budget",
    "entry": "Budget",
    "mid": "Mid-range",
    "mid_range": "Mid-range",
    "mid-range": "Mid-range",
    "premium": "Premium",
    "luxury": "Luxury",
    "ultra": "Ultra Luxury",
    "ultra-luxury": "Ultra Luxury",
    "ultra_luxury": "Ultra Luxury",
    "ultra luxury": "Ultra Luxury",
}
TIER_MAP_ID = {
    "Budget": "budget",
    "Mid-range": "mid",
    "Premium": "premium",
    "Luxury": "luxury",
    "Ultra Luxury": "ultra",
}

def canon_tier_name(x: str) -> str:
    s = str(x or "").strip()
    if not s:
        return ""
    key = s.lower().replace(" ", "_").replace("__", "_").replace("-", "_")
    return TIER_MAP_NAME.get(key, s)

def tier_id_from_name(name: str) -> str:
    return TIER_MAP_ID.get(canon_tier_name(name), "")

def to_ml(size_val):
    """Accepts '750', '75cl', '1.5L', 750 etc. -> returns int ml or np.nan"""
    if size_val is None:
        return np.nan
    s = str(size_val).strip().lower()
    try:
        if s.endswith("ml"):
            return int(float(s[:-2]))
        if s.endswith("cl"):
            return int(float(s[:-2]) * 10)
        if s.endswith("l"):
            return int(float(s[:-1]) * 1000)
        # plain number: if < 100 -> assume cl, else ml
        v = float(s)
        return int(v * 10) if v < 100 else int(v)
    except Exception:
        return np.nan

def ensure_cols(df: pd.DataFrame, cols: dict):
    """Ensure columns exist with default; cols = {name: default}"""
    for c, default in cols.items():
        if c not in df.columns:
            df[c] = default
    return df

# ---------- Default region/type from any top picks (optional) ----------
default_region = "Unknown"
default_type   = "Unknown"

if 'top3_by_type' in globals() and isinstance(top3_by_type, pd.DataFrame) and not top3_by_type.empty:
    top_rec = top3_by_type.iloc[0]
    wine_id = str(top_rec.get('id', "")).strip()
    default_type = str(top_rec.get('full_type', 'Unknown')) or 'Unknown'
    try:
        stock_df_ids = stock_df.copy()
        stock_df_ids['id'] = stock_df_ids['id'].astype(str)
        match_row = stock_df_ids.loc[stock_df_ids['id'] == wine_id, 'region_group']
        if not match_row.empty:
            default_region = str(match_row.iloc[0]) or "Unknown"
    except Exception:
        pass
else:
    print("⚠️ top3_by_type is empty — using 'Unknown' defaults for region/type.")

# Build seg_df filling missing values with defaults (but not overwriting existing)
seg_df = client_pref_df.copy()
seg_df['region_group'] = seg_df.get('region_group', pd.Series(index=seg_df.index, dtype='object')).fillna(default_region)
seg_df['full_type']    = seg_df.get('full_type',    pd.Series(index=seg_df.index, dtype='object')).fillna(default_type)

# ---------- Load UI filters (file -> env fallback -> defaults) ----------
filters_path = Path("notebooks") / "filters.json"
filters = {}
try:
    if filters_path.exists():
        filters = json.loads(filters_path.read_text(encoding="utf-8")) or {}
    elif os.getenv("FILTER_INPUTS"):
        filters = json.loads(os.getenv("FILTER_INPUTS"))
except Exception as e:
    print(f"⚠️ Failed to read filters.json/env: {e}")
finally:
    if not filters:
        filters = {
            "last_stock": False,
            "seasonality_boost": False,
            "wine_type": None,
            "bottle_size": None,
            "price_tier_bucket": "",
            "price_tiers": [],
            "loyalty_levels": []
        }

print("🔍 Effective UI filters:\n", json.dumps(filters, indent=2))

# ---------- Normalize stock needed columns ----------
stock_df = stock_df.copy()
stock_df['id']    = stock_df.get('id', "").astype(str)
stock_df['stock'] = pd.to_numeric(stock_df.get('stock', 0), errors='coerce').fillna(0).astype(int)
stock_df['price_tier'] = stock_df.get('price_tier', "").map(canon_tier_name)
stock_df['price_tier_id'] = stock_df['price_tier'].map(tier_id_from_name)

if 'bottle_size_ml' not in stock_df.columns:
    if 'size' in stock_df.columns:
        stock_df['bottle_size_ml'] = stock_df['size'].apply(to_ml)
    elif 'size_cl' in stock_df.columns:
        stock_df['bottle_size_ml'] = pd.to_numeric(stock_df['size_cl'], errors='coerce') * 10
    else:
        stock_df['bottle_size_ml'] = np.nan

stock_df = ensure_cols(stock_df, {'occasion': 'Unknown'})

# ---------- Apply global UI filters ----------
_selected_tiers = set()
if filters.get("price_tiers"):
    _selected_tiers.update([canon_tier_name(t) for t in filters["price_tiers"] if t])
if filters.get("price_tier_bucket"):
    _selected_tiers.add(canon_tier_name(filters["price_tier_bucket"]))

if _selected_tiers:
    stock_df = stock_df[stock_df['price_tier'].isin(_selected_tiers)].copy()

_selected_loyalties = {str(x).strip().lower() for x in (filters.get("loyalty_levels") or []) if str(x).strip()}
if _selected_loyalties and 'loyalty_level' in seg_df.columns:
    seg_df = seg_df[seg_df['loyalty_level'].str.lower().isin(_selected_loyalties)].copy()

last_stock_flag    = bool(filters.get("last_stock", False))
seasonality_flag   = bool(filters.get("seasonality_boost", False))
selected_type      = filters.get("wine_type", None)
selected_size_raw  = filters.get("bottle_size", None)
try:
    selected_size_ml = int(selected_size_raw) if (selected_size_raw is not None and str(selected_size_raw).isdigit()) else None
except Exception:
    selected_size_ml = None

# ---------- Weekday policies ----------
WEEKDAY_OCCASION = {
    "Monday": "Casual", "Tuesday": "Casual", "Wednesday": "Dinner",
    "Thursday": "Dinner", "Friday": "Party", "Saturday": "Gifting", "Sunday": "Dinner"
}
WEEKDAY_TIERS = {
    "Monday":   ["Budget", "Mid-range", "Premium"],
    "Tuesday":  ["Mid-range", "Premium"],
    "Wednesday":["Premium", "Luxury"],
    "Thursday": ["Premium", "Luxury"],
    "Friday":   ["Luxury", "Ultra Luxury"],
    "Saturday": ["Luxury", "Ultra Luxury"],
    "Sunday":   ["Budget", "Premium", "Luxury"],
}

# ---------- Selector ----------
def get_seasonal_wines_modular(stock_df, seg_df, last_stock_flag=False, seasonality=False,
                               selected_type=None, selected_size_ml=None, num_per_day=5):
    df = stock_df.copy()

    # Type filter
    if selected_type and str(selected_type).strip().lower() not in ("all", ""):
        df = df[df['full_type'].astype(str).str.casefold() == str(selected_type).casefold()]

    # Size filter (ml)
    if selected_size_ml is not None and not np.isnan(selected_size_ml):
        df = df[np.isclose(pd.to_numeric(df['bottle_size_ml'], errors='coerce'), selected_size_ml, equal_nan=False)]

    # Seasonality: within ~last year → next week window (based on OMT last offer date)
    if seasonality:
        last_year = datetime.today() - timedelta(days=365)
        next_week = last_year + timedelta(days=7)
        df['OMT last offer date'] = pd.to_datetime(df.get('OMT last offer date', pd.NaT), errors='coerce')
        df = df[df['OMT last offer date'].between(last_year, next_week)]

    # Stock filters
    if last_stock_flag:
        df = df[df['stock'] < 10]
    df = df[df['stock'] >= 3]

    # Segment match score (region+type)
    seg = seg_df[['region_group','full_type']].dropna()
    def match_score(row):
        return int(((seg['region_group'] == row['region_group']) & (seg['full_type'] == row['full_type'])).sum()) if not seg.empty else 0
    df['segment_score'] = df.apply(match_score, axis=1) if not df.empty else 0

    ultra_cap = 2
    ultra_used = 0

    calendar = {}

    for day in DAYS:
        allowed_tiers = [canon_tier_name(t) for t in WEEKDAY_TIERS.get(day, [])]
        occasion = WEEKDAY_OCCASION.get(day, "Casual")

        pool = df.copy()

        # Step 1: match occasion + tier
        cand = pool[(pool['occasion'].astype(str).str.casefold() == occasion.casefold()) &
                    (pool['price_tier'].isin(allowed_tiers))]

        # Ultra cap per week
        if ultra_used >= ultra_cap:
            cand = cand[cand['price_tier'] != "Ultra Luxury"]

        sel = cand.sort_values(['segment_score','stock'], ascending=[False, False]).head(num_per_day)

        # Step 2: relax occasion
        if sel.empty:
            cand2 = pool[pool['price_tier'].isin(allowed_tiers)]
            if ultra_used >= ultra_cap:
                cand2 = cand2[cand2['price_tier'] != "Ultra Luxury"]
            sel = cand2.sort_values(['segment_score','stock'], ascending=[False, False]).head(num_per_day)

        # Step 3: seasonal fallback
        if sel.empty:
            seasonal_only = pool.copy()
            seasonal_only['OMT last offer date'] = pd.to_datetime(seasonal_only.get('OMT last offer date', pd.NaT), errors='coerce')
            last_year = datetime.today() - timedelta(days=365)
            next_week = last_year + timedelta(days=7)
            fallback = seasonal_only[seasonal_only['OMT last offer date'].between(last_year, next_week)]
            sel = fallback.sort_values('stock', ascending=False).head(num_per_day)

        # Step 4: top stocked fallback
        if sel.empty:
            sel = pool.sort_values('stock', ascending=False).head(num_per_day)

        # track ultra usage
        if not sel.empty and "Ultra Luxury" in sel['price_tier'].values:
            ultra_used = min(ultra_cap, ultra_used + (sel['price_tier'].eq("Ultra Luxury").sum()))

        calendar[day] = sel

    return calendar

# ---------- UI mapping ----------
def to_ui_item(row: pd.Series) -> dict:
    return {
        'id': str(row.get('id', '')),
        'wine': row.get('wine', 'Unknown'),
        'name': row.get('wine', 'Unknown'),
        'vintage': str(row.get('vintage', 'NV')) if pd.notna(row.get('vintage', '')) else 'NV',
        'full_type': row.get('full_type', 'Unknown'),
        'region_group': row.get('region_group', 'Unknown'),
        'stock': int(pd.to_numeric(row.get('stock', 0), errors='coerce') if row.get('stock', 0) is not None else 0),
        'price_tier': canon_tier_name(row.get('price_tier', '')),
        'price_tier_id': tier_id_from_name(row.get('price_tier', '')),
        'match_quality': row.get('match_quality', 'Auto'),
        'avg_cpi_score': float(pd.to_numeric(row.get('avg_cpi_score', 0), errors='coerce')) if pd.notna(row.get('avg_cpi_score', np.nan)) else 0.0,
        'locked': bool(row.get('locked', False))
    }

def get_ui_data(top_n_recs: pd.DataFrame, weekly_wines: dict, top_wine_counts: pd.DataFrame,
                client_prefs: pd.DataFrame, locked_df: pd.DataFrame | None = None,
                winner_boxes: dict | None = None):
    calendar_data = {d: [] for d in DAYS}

    # 1) Locked first
    if isinstance(locked_df, pd.DataFrame) and not locked_df.empty:
        for day in DAYS:
            for _, r in locked_df.loc[locked_df['day']==day].iterrows():
                calendar_data[day].append(to_ui_item(r))

    # 1.5) Winners next (dedicated 3 boxes). Skip duplicates, respect NUM_SLOTS
    if isinstance(winner_boxes, dict) and winner_boxes:
        for day in DAYS:
            have = {str(it.get("id","")) for it in calendar_data[day]}
            for it in (winner_boxes.get(day) or []):
                if len(calendar_data[day]) >= NUM_SLOTS:
                    break
                rid = str(it.get("id","")).strip()
                if rid and rid not in have:
                    # ensure basic schema; Cell 10 will normalize tier/ids again
                    it.setdefault("match_quality", "Winner")
                    it.setdefault("locked", True)
                    it.setdefault("avg_cpi_score", 0.0)
                    calendar_data[day].append(it)
                    have.add(rid)

    # 2) Auto picks (skip duplicates)
    for day, df_day in (weekly_wines or {}).items():
        if isinstance(df_day, pd.DataFrame) and not df_day.empty:
            have = {str(it.get("id","")) for it in calendar_data[day]}
            for _, r in df_day.iterrows():
                rid = str(r.get("id",""))
                if rid and rid not in have:
                    calendar_data[day].append(to_ui_item(r))
                    have.add(rid)

    # CPI preview (avg of attached)
    cpi_score = 0.0
    try:
        flat = [it.get("avg_cpi_score", 0) for arr in calendar_data.values() for it in arr]
        if flat:
            cpi_score = float(pd.to_numeric(pd.Series(flat), errors="coerce").fillna(0).mean())
    except Exception:
        pass

    return {
        'weekly_calendar': calendar_data,
        'cpi_score': round(cpi_score, 2),
        'client_prefs': client_prefs.to_dict('records'),
        'top_recommendations': []
    }

# --- Ensure at least N budget offers across the week (non-destructive to locked/winners) ---
def _inject_budget_offers(calendar_data: dict, stock_df: pd.DataFrame, min_budget: int = 3) -> dict:
    cal = {d: list(calendar_data.get(d, [])) for d in DAYS}

    def _canon_tier(x):
        return canon_tier_name(x) if isinstance(x, str) else ""

    # Current IDs + budget count
    current_ids = {str(it.get("id", "")).strip()
                   for day in DAYS for it in cal.get(day, []) if isinstance(it, dict)}
    def _is_budget(it): return _canon_tier(it.get("price_tier", "")) == "Budget"
    have_budget = sum(1 for day in DAYS for it in cal.get(day, []) if _is_budget(it))
    need = max(0, min_budget - have_budget)
    if need == 0:
        return cal

    # Candidate pool: Budget, in stock, not used yet
    s = stock_df.copy()
    s["id"] = s.get("id", "").astype(str).str.strip()
    s["price_tier"] = s.get("price_tier", "").map(canon_tier_name)
    s["stock"] = pd.to_numeric(s.get("stock", 0), errors="coerce").fillna(0).astype(int)
    s["avg_cpi_score"] = pd.to_numeric(s.get("avg_cpi_score", 0), errors="coerce").fillna(0.0)

    cand = (s[(s["price_tier"] == "Budget") & (s["stock"] > 0) & (~s["id"].isin(current_ids))]
            .sort_values(["avg_cpi_score", "stock"], ascending=[False, False])
            .head(need)
            .to_dict("records"))

    # Preferred placement order: Mon → Tue → Sun → Thu → Fri → Sat → Wed
    target_days = ["Monday", "Tuesday", "Sunday", "Thursday", "Friday", "Saturday", "Wednesday"]

    def _is_protected(it: dict) -> bool:
        if not isinstance(it, dict): return False
        if bool(it.get("locked", False)): return True
        tag = str(it.get("match_quality", "")).strip().lower()
        return tag in {"winner", "cadence"}

    for r in cand:
        card = to_ui_item(pd.Series(r))  # shape the UI fields consistently
        placed = False

        # 1) Try to append where there is room
        for day in target_days:
            day_list = cal.setdefault(day, [])
            if len(day_list) < NUM_SLOTS:
                day_list.append(card)
                placed = True
                break

        # 2) If no room, replace first non-locked, non-winner/cadence item
        if not placed:
            for day in target_days:
                day_list = cal.setdefault(day, [])
                replace_idx = next((i for i, it in enumerate(day_list) if not _is_protected(it)), None)
                if replace_idx is not None:
                    day_list[replace_idx] = card
                    placed = True
                    break
        # If still not placed, we silently skip (calendar is fully protected)

    return cal

# ---------- Build weekly selection ----------
IRON_DATA_PATH = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)

weekly_wines = get_seasonal_wines_modular(
    stock_df, seg_df,
    last_stock_flag=last_stock_flag,
    seasonality=seasonality_flag,
    selected_type=selected_type,
    selected_size_ml=selected_size_ml,
    num_per_day=NUM_SLOTS
)

ui_output_data = get_ui_data(
    top_n_recs=pd.DataFrame(),
    weekly_wines=weekly_wines,
    top_wine_counts=pd.DataFrame(),
    client_prefs=client_pref_df,
    locked_df=locked_df,
    winner_boxes=winner_boxes
)

# NEW: ensure at least 3 Budget offers without clashing with locked/winner/cadence
ui_output_data['weekly_calendar'] = _inject_budget_offers(
    ui_output_data['weekly_calendar'], stock_df, min_budget=3
)

# Normalize & cap to NUM_SLOTS
calendar_norm = {d: (ui_output_data['weekly_calendar'].get(d, []) or [])[:NUM_SLOTS] for d in DAYS}

# ---------- Write artifacts ----------
(IRON_DATA_PATH / "weekly_campaign_schedule.json").write_text(
    json.dumps(calendar_norm, indent=4), encoding="utf-8"
)
(IRON_DATA_PATH / f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.json").write_text(
    json.dumps(calendar_norm, indent=4), encoding="utf-8"
)
# Legacy (week only)
(IRON_DATA_PATH / f"weekly_campaign_schedule_week_{sel_week}.json").write_text(
    json.dumps(calendar_norm, indent=4), encoding="utf-8"
)
# UI-preferred artifact
(IRON_DATA_PATH / f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.ui.json").write_text(
    json.dumps({"weekly_calendar": calendar_norm}, indent=2), encoding="utf-8"
)

# PKLs
pd.to_pickle(calendar_norm, IRON_DATA_PATH / "weekly_campaign_schedule.pkl")
pd.to_pickle(calendar_norm, IRON_DATA_PATH / f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.pkl")
pd.to_pickle(calendar_norm, IRON_DATA_PATH / f"weekly_campaign_schedule_week_{sel_week}.pkl")

# Leads (present if previous logic produced it; otherwise sane default)
leads_payload = ui_output_data.get("leads_campaigns", {"TueWed": [], "ThuFri": []})
(IRON_DATA_PATH / "leads_campaigns.json").write_text(json.dumps(leads_payload, indent=2), encoding="utf-8")
(IRON_DATA_PATH / f"leads_campaigns_{sel_year}_week_{sel_week}.json").write_text(json.dumps(leads_payload, indent=2), encoding="utf-8")

# Index (nested by year→week)
index_path = IRON_DATA_PATH / "schedule_index.json"
try:
    idx = json.loads(index_path.read_text(encoding="utf-8")) if index_path.exists() else {}
except Exception:
    idx = {}
idx.setdefault(str(sel_year), {})[str(sel_week)] = {
    "json": f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.json",
    "json_ui": f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.ui.json",
    "pkl": f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.pkl",
    "leads_json": f"leads_campaigns_{sel_year}_week_{sel_week}.json",
    "updated_at": datetime.now().isoformat(timespec="seconds")
}
idx["_latest_year"] = str(sel_year)
idx["_latest_week"] = str(sel_week)
(IRON_DATA_PATH / "schedule_index.json").write_text(json.dumps(idx, indent=2), encoding="utf-8")

print(f"✅ Saved year+week UI files for {sel_year}-W{sel_week} in {IRON_DATA_PATH}")


⚠️ top3_by_type is empty — using 'Unknown' defaults for region/type.
🔍 Effective UI filters:
 {
  "last_stock": false,
  "seasonality_boost": false,
  "wine_type": null,
  "bottle_size": null,
  "price_tier_bucket": "",
  "price_tiers": [],
  "loyalty_levels": []
}
✅ Saved year+week UI files for 2025-W38 in C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA


In [13]:
# --- CELL 13: Build weekly "Leads" campaigns (Tue–Wed & Thu–Fri) ---
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import json

# Ensure IRON_DATA_PATH exists (fallback to OUTPUT_PATH or default OneDrive path)
if 'IRON_DATA_PATH' not in globals():
    if 'OUTPUT_PATH' in globals():
        IRON_DATA_PATH = OUTPUT_PATH
    else:
        IRON_DATA_PATH = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)

def _canon_tier_name(x: str) -> str:
    s = str(x or "").strip().lower().replace("-", " ").replace("_", " ")
    if not s: return ""
    if "ultra" in s: return "Ultra Luxury"
    if "luxury" in s: return "Luxury"
    if "premium" in s: return "Premium"
    if "mid" in s: return "Mid-range"
    if "budget" in s or "<" in s or "cheap" in s: return "Budget"
    return s.title()

def _load_inventory(iron_path: Path) -> pd.DataFrame:
    # Prefer the most enriched inventory if present
    candidates = [
        iron_path / "stock_df_with_seasonality.pkl",
        iron_path / "stock_df_final.pkl",
    ]
    src = next((p for p in candidates if p.exists()), None)
    if not src:
        return pd.DataFrame(columns=["id","wine","vintage","full_type","region_group","stock","price_tier","avg_cpi_score"])
    df = pd.read_pickle(src)

    # Normalize columns we rely on
    for c in ("id","wine","vintage","full_type","region_group","stock","price_tier","price","avg_cpi_score"):
        if c not in df.columns:
            df[c] = np.nan

    df["id"] = df["id"].astype("string").fillna("").str.replace(r"\.0$", "", regex=True).str.strip()
    df["wine"] = df["wine"].astype("string").fillna("").str.strip()
    df["vintage"] = df["vintage"].astype("string").fillna("").str.strip().replace({"": "NV","nan":"NV"})
    df["full_type"] = df["full_type"].astype("string").fillna("").str.strip()
    df["region_group"] = df["region_group"].astype("string").fillna("").str.strip()
    df["stock"] = pd.to_numeric(df["stock"], errors="coerce").fillna(0).astype(int)

    # Price tier: prefer provided; else derive from price if present
    df["price_tier"] = df["price_tier"].astype("string").fillna("").map(_canon_tier_name)
    if df["price_tier"].eq("").any() and "price" in df.columns:
        price_num = pd.to_numeric(df["price"], errors="coerce")
        def _bucket(p):
            if pd.isna(p): return ""
            if p < 50: return "Budget"
            if p < 100: return "Mid-range"
            if p < 200: return "Premium"
            if p < 500: return "Luxury"
            return "Ultra Luxury"
        mask = df["price_tier"].eq("")
        df.loc[mask, "price_tier"] = price_num.map(_bucket)

    # keep only rows with something to offer
    df = df[df["stock"] > 0].copy()
    return df

def _attach_last_campaign(inv: pd.DataFrame, history: pd.DataFrame | None) -> pd.DataFrame:
    """
    Attach last_campaign_date to inventory rows. Uses history_df when provided;
    otherwise falls back to OUTPUT/IRON campaign_index.json if available.
    """
    inv = inv.copy()
    inv["last_campaign_date"] = pd.NaT

    # 1) If a rich history_df is available
    if history is not None and not history.empty and {"id","wine","vintage","schedule_dt"}.issubset(history.columns):
        hist = history.copy()
        hist["id"] = hist["id"].astype("string").fillna("").str.replace(r"\.0$","",regex=True).str.strip()
        hist["wine"] = hist["wine"].astype("string").fillna("").str.strip()
        hist["vintage"] = hist["vintage"].astype("string").fillna("").str.strip().replace({"": "NV","nan":"NV"})
        hist["schedule_dt"] = pd.to_datetime(hist["schedule_dt"], errors="coerce")

        last_seen = (
            hist.dropna(subset=["schedule_dt"])
                .groupby(["id","wine","vintage"], dropna=False)["schedule_dt"]
                .max()
                .reset_index()
                .rename(columns={"schedule_dt":"last_campaign_date"})
        )
        return inv.merge(last_seen, how="left", on=["id","wine","vintage"])

    # 2) Fallback: campaign_index.json (fast, id-based)
    idx_path = IRON_DATA_PATH / "campaign_index.json"
    if not idx_path.exists() and 'OUTPUT_PATH' in globals():
        alt = OUTPUT_PATH / "campaign_index.json"
        if alt.exists():
            idx_path = alt

    if idx_path.exists():
        try:
            ci = json.loads(idx_path.read_text(encoding="utf-8"))
            by_id = ci.get("by_id", {})
            id_map = {str(k): v.get("last_campaign_date") for k, v in by_id.items()}
            inv["last_campaign_date"] = inv["id"].map(id_map)
            inv["last_campaign_date"] = pd.to_datetime(inv["last_campaign_date"], errors="coerce")
        except Exception:
            pass

    return inv

def _pick_one(df, sort_cols, asc):
    if df.empty: return pd.DataFrame(columns=df.columns)
    return df.sort_values(sort_cols, ascending=asc, na_position="last").head(1)

def _pick_varied(df, used_ids, k=2):
    pool = df[~df["id"].isin(used_ids)].copy()
    picks = []

    # prefer different full_type across picks if possible
    type_order = ["Sparkling","White","Rosé","Rose","Red","Dessert"]
    for t in type_order:
        cand = pool[pool["full_type"].str.contains(t, case=False, na=False)]
        got = _pick_one(cand, ["stock","last_campaign_date"], [False, True])
        if not got.empty:
            picks.append(got)
            pool = pool[~pool["id"].isin(got["id"])]
        if len(picks) >= k:
            break

    # fallback if we didn’t get enough
    if len(picks) < k and not pool.empty:
        extra = pool.sort_values(["stock","last_campaign_date"], ascending=[False, True], na_position="last").head(k - len(picks))
        if not extra.empty:
            picks.append(extra)

    return pd.concat(picks, ignore_index=True) if picks else pd.DataFrame(columns=df.columns)

def build_weekly_leads(iron_path: Path, history_df: pd.DataFrame | None) -> dict:
    inv = _load_inventory(iron_path)
    inv = _attach_last_campaign(inv, history_df)

    # 1) Budget (high stock)
    budget = inv[inv["price_tier"].eq("Budget")]
    lead_budget = _pick_one(budget, ["stock","last_campaign_date"], [False, True])

    # 2) Ultra Luxury (low stock)
    ultra = inv[inv["price_tier"].eq("Ultra Luxury")]
    lead_ultra = _pick_one(ultra, ["stock","last_campaign_date"], [True, True])

    # 3–4) Two varied items (not already chosen)
    used_ids = pd.concat([lead_budget, lead_ultra])["id"].astype(str).tolist() if not pd.concat([lead_budget, lead_ultra]).empty else []
    varied = _pick_varied(inv, used_ids, k=2)

    leads_df = pd.concat([lead_budget, lead_ultra, varied], ignore_index=True)
    if leads_df.empty:
        return {"TueWed": [], "ThuFri": []}

    def _row_to_item(r):
        return {
            "id": str(r.get("id","")),
            "wine": r.get("wine") or "Unknown",
            "name": r.get("wine") or "Unknown",
            "vintage": str(r.get("vintage","NV")) if str(r.get("vintage","")).strip() else "NV",
            "full_type": r.get("full_type") or "Unknown",
            "region_group": r.get("region_group") or "Unknown",
            "stock": int(pd.to_numeric(r.get("stock", 0), errors="coerce") or 0),
            "price_tier": r.get("price_tier") or "",
            "match_quality": "Lead",
            "avg_cpi_score": float(pd.to_numeric(r.get("avg_cpi_score", 0), errors="coerce") or 0),
            "last_campaign_date": (
                r.get("last_campaign_date").isoformat()
                if pd.notna(r.get("last_campaign_date")) else None
            ),
            # UI helpers:
            "locked": False,
            "span_days": 2,             # 2-day “Leads” box
            "campaign_tag": "leads"
        }

    items = [ _row_to_item(r) for _, r in leads_df.iterrows() ]
    # Same set in both boxes (simple + consistent)
    return {"TueWed": items, "ThuFri": items}

# Get history_df if available; otherwise pass None (we’ll fall back to campaign_index.json)
_hist = history_df if 'history_df' in globals() else None
try:
    leads_campaigns = build_weekly_leads(IRON_DATA_PATH, _hist)
except Exception as e:
    print(f"⚠️ build_weekly_leads failed: {e}")
    leads_campaigns = {"TueWed": [], "ThuFri": []}

# Persist year+week leads alongside generic (for fast API fetches)
try:
    y, w = int(calendar_year), int(week_number)
except Exception:
    y, w = datetime.now().year, datetime.now().isocalendar().week

lp = IRON_DATA_PATH
lp.mkdir(parents=True, exist_ok=True)
(lp / "leads_campaigns.json").write_text(json.dumps(leads_campaigns, indent=2), encoding="utf-8")
(lp / f"leads_campaigns_{y}_week_{w}.json").write_text(json.dumps(leads_campaigns, indent=2), encoding="utf-8")
print(f"📣 Leads saved for {y}-W{w}")

# Attach to in-memory UI bundle if present
if "ui_output_data" not in locals():
    ui_output_data = {}
ui_output_data["leads_campaigns"] = leads_campaigns

# (Removed undefined atomic_write_* calls and the incorrect 'leads_payload' reference.)


📣 Leads saved for 2025-W38


In [30]:
# --- CELL 14: Flask handoff & week outputs (atomic + robust, preserves overflow) ---
import os
import json
from datetime import datetime, timezone, timedelta
from pathlib import Path
import pandas as pd
import numpy as np

# === Safety Check ===
if "ui_output_data" not in locals():
    raise RuntimeError("❌ ui_output_data not found. Make sure the previous cell populated it.")

# === Status bridge (prefer package import; fallback to none) ===
try:
    from utils.notebook_status import update_status  # when run by Flask
except Exception:
    try:
        from notebook_status import update_status     # when run locally
    except Exception as e:
        print(f"⚠️ update_status unavailable: {e}")
        update_status = None

# === Paths / Week ===
IRON_DATA_PATH = Path(
    globals().get("OUTPUT_PATH")
    or globals().get("output_path")
    or os.getenv("OUTPUT_PATH")
    or os.getenv("IRON_DATA")
    or (Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA")
)
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)

try:
    week_number = int(globals().get("week_number", os.getenv("WEEK_NUMBER", datetime.now().isocalendar().week)))
except Exception:
    week_number = datetime.now().isocalendar().week

# === Helpers ===
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

TIER_MAP_NAME = {
    "budget":"Budget","entry":"Budget",
    "mid":"Mid-range","mid_range":"Mid-range","mid-range":"Mid-range",
    "premium":"Premium",
    "luxury":"Luxury",
    "ultra":"Ultra Luxury","ultra_luxury":"Ultra Luxury","ultra-luxury":"Ultra Luxury","ultra luxury":"Ultra Luxury",
}
TIER_MAP_ID = {"Budget":"budget","Mid-range":"mid","Premium":"premium","Luxury":"luxury","Ultra Luxury":"ultra"}

def canon_tier_name(x: str) -> str:
    s = str(x or "").strip()
    if not s: return ""
    key = s.lower().replace("-", "_").replace(" ", "_")
    return TIER_MAP_NAME.get(key, s)

def tier_id_from_name(name: str) -> str:
    return TIER_MAP_ID.get(canon_tier_name(name), "")

def _to_int(x, default=0):
    try:
        return int(float(x))
    except Exception:
        try:
            return int(pd.to_numeric(x, errors="coerce").fillna(default))
        except Exception:
            return default

def _now_iso():
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def atomic_write_text(path: Path, text: str):
    tmp = path.with_suffix(path.suffix + ".tmp")
    tmp.write_text(text, encoding="utf-8")
    os.replace(tmp, path)

def atomic_write_pkl(path: Path, obj):
    tmp = path.with_suffix(path.suffix + ".tmp")
    pd.to_pickle(obj, tmp)
    os.replace(tmp, path)

def _norm_item(it: dict) -> dict:
    it = it or {}
    price_tier = canon_tier_name(it.get("price_tier", it.get("price_tier_bucket", "")))
    return {
        "id":              str(it.get("id", "")),
        "wine":            it.get("wine") or it.get("name") or "Unknown",
        "name":            it.get("name") or it.get("wine") or "Unknown",
        "vintage":         str(it.get("vintage", "NV")) if str(it.get("vintage","")).strip() else "NV",
        "full_type":       it.get("full_type", "Unknown"),
        "region_group":    it.get("region_group", "Unknown"),
        "stock":           _to_int(it.get("stock", it.get("stock_count", 0))),
        "price_tier":      price_tier,
        "price_tier_id":   tier_id_from_name(price_tier),
        "match_quality":   it.get("match_quality", "Auto"),
        "avg_cpi_score":   float(pd.to_numeric(it.get("avg_cpi_score", it.get("cpi_score", 0)), errors="coerce") or 0),
        "locked":          bool(it.get("locked", False)),
    }

def normalize_calendar_with_overflow(calendar_raw: dict, num_slots: int = 5,
                                     allow_overflow_days: set = {"Wednesday"}):
    """
    Accepts either:
      - flat: {day: [items]}
      - structured: {day: {"main":[...], "overflow":[...]}}
    Returns (calendar_main, calendar_overflow) both normalized.
    """
    cal_main = {d: [] for d in DAYS}
    cal_over = {d: [] for d in DAYS}
    if not isinstance(calendar_raw, dict):
        return cal_main, cal_over

    for day in DAYS:
        node = calendar_raw.get(day) or []
        # support structured input
        if isinstance(node, dict) and ("main" in node or "overflow" in node):
            main = node.get("main") or []
            over = node.get("overflow") or []
            main_norm = [_norm_item(x) for x in main]
            over_norm = [_norm_item(x) for x in over]
            # enforce cap only on main; preserve provided overflow
            cal_main[day] = main_norm[:num_slots]
            cal_over[day] = over_norm
        else:
            items = [_norm_item(x) for x in (node if isinstance(node, list) else [])]
            if day in (allow_overflow_days or set()):
                cal_main[day] = items[:num_slots]
                cal_over[day] = items[num_slots:]  # preserve extras as overflow
            else:
                cal_main[day] = items[:num_slots]
                cal_over[day] = []  # drop extras for non-overflow days
    return cal_main, cal_over

# === Normalize payload from the previous cell(s) ===
calendar_raw = ui_output_data.get("weekly_calendar", {}) or {}
cal_main, cal_over = normalize_calendar_with_overflow(calendar_raw, num_slots=5, allow_overflow_days={"Wednesday"})

# Leads payload produced earlier
leads_payload = ui_output_data.get("leads_campaigns", {"TueWed": [], "ThuFri": []})

# Optional summary for status callback
summary = {
    "week_number": week_number,
    "updated_at": _now_iso(),
    "days": {d: len(cal_main.get(d, [])) for d in DAYS},
    "overflow": {d: len(cal_over.get(d, [])) for d in DAYS},
    "cpi_score": ui_output_data.get("cpi_score", None),
    "top_recommendations": ui_output_data.get("top_recommendations", []),
    "leads_counts": {k: len(v or []) for k, v in (leads_payload or {}).items()}
}

# === Save (atomic) ===
json_generic = IRON_DATA_PATH / "weekly_campaign_schedule.json"                     # flat/main only (compat)
pkl_generic  = IRON_DATA_PATH / "weekly_campaign_schedule.pkl"
json_week    = IRON_DATA_PATH / f"weekly_campaign_schedule_week_{week_number}.json"
pkl_week     = IRON_DATA_PATH / f"weekly_campaign_schedule_week_{week_number}.pkl"

# Structured artifact that preserves overflow
json_struct  = IRON_DATA_PATH / "weekly_campaign_schedule_structured.ui.json"
json_struct_w= IRON_DATA_PATH / f"weekly_campaign_schedule_structured_week_{week_number}.ui.json"

# Dedicated leads files
leads_json_generic = IRON_DATA_PATH / "leads_campaigns.json"
leads_json_week    = IRON_DATA_PATH / f"leads_campaigns_week_{week_number}.json"

index_path   = IRON_DATA_PATH / "schedule_index.json"

try:
    # Write calendar artifacts (main-only for legacy/UI)
    atomic_write_text(json_generic, json.dumps(cal_main, indent=4))
    atomic_write_text(json_week,    json.dumps(cal_main, indent=4))
    atomic_write_pkl(pkl_generic, cal_main)
    atomic_write_pkl(pkl_week,    cal_main)

    # Write structured artifacts (main + overflow preserved)
    atomic_write_text(json_struct,  json.dumps({"weekly_calendar": cal_main, "overflow": cal_over}, indent=2))
    atomic_write_text(json_struct_w, json.dumps({"weekly_calendar": cal_main, "overflow": cal_over}, indent=2))

    # Write leads artifacts
    atomic_write_text(leads_json_generic, json.dumps(leads_payload, indent=2))
    atomic_write_text(leads_json_week,    json.dumps(leads_payload, indent=2))

    # Maintain a simple index for the server (keeps legacy; adds structured)
    try:
        idx = json.loads(index_path.read_text(encoding="utf-8")) if index_path.exists() else {}
    except Exception:
        idx = {}
    idx[str(week_number)] = {
        "json": json_week.name,
        "pkl":  pkl_week.name,
        "leads_json": leads_json_week.name,
        "structured": json_struct_w.name,
        "updated_at": summary["updated_at"]
    }
    idx["_latest"] = str(week_number)
    atomic_write_text(index_path, json.dumps(idx, indent=2))

    # Console summary
    main_counts = {d: len(cal_main[d]) for d in DAYS}
    over_counts = {d: len(cal_over[d]) for d in DAYS}
    print(f"✅ UI JSON and PKL saved atomically for week {week_number}")
    print("🧩 Calendar composed (main vs overflow):")
    for d in DAYS:
        print(f"  • {d}: {main_counts[d]} main, {over_counts[d]} overflow")
    print("🎯 CPI score:", ui_output_data.get('cpi_score', 'N/A'))
    print("📣 Leads (counts):", summary["leads_counts"])

except Exception as e:
    print(f"❌ An error occurred while saving UI files: {e}")

# === Report back to Flask UI (if available) ===
if callable(update_status):
    try:
        _nb_name = globals().get("NOTEBOOK_NAME") or os.getenv("NOTEBOOK_NAME") or "AVU_schedule_only.ipynb"
        update_status(
            progress=100,
            message="✅ Notebook finished.",
            state="completed",
            done=True,
            notebook=_nb_name,
            meta=summary
        )
    except Exception as e:
        print(f"⚠️ update_status failed: {e}")


⚠️ update_status unavailable: No module named 'notebook_status'
✅ UI JSON and PKL saved atomically for week 38
🧩 Calendar composed (main vs overflow):
  • Monday: 5 main, 0 overflow
  • Tuesday: 5 main, 0 overflow
  • Wednesday: 5 main, 1 overflow
  • Thursday: 5 main, 0 overflow
  • Friday: 5 main, 0 overflow
  • Saturday: 5 main, 0 overflow
  • Sunday: 5 main, 0 overflow
🎯 CPI score: 0.18
📣 Leads (counts): {'TueWed': 0, 'ThuFri': 0}


In [31]:
# --- CELL 15: Generate a file readable by Power BI (from normalized stock)
# --- then PowerBI visualization will be on the UI
import os
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd

# 1) Get the normalized stock frame
try:
    df = stock_df.copy()  # from previous cells
except NameError:
    df_path = OUTPUT_PATH / "stock_df_final.pkl"
    if not df_path.exists():
        raise FileNotFoundError(f"❌ Could not find {df_path}. Run earlier cells first.")
    df = pd.read_pickle(df_path)

# 2) Ensure required columns exist
for col in ["price_tier", "CHF Price", "vintage", "id", "wine", "region_group", "full_type", "stock"]:
    if col not in df.columns:
        df[col] = np.nan

# Normalize IDs and numeric CHF price
df["id"] = df["id"].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
df["CHF Price"] = pd.to_numeric(df["CHF Price"], errors="coerce")
df["stock"] = pd.to_numeric(df["stock"], errors="coerce")

# 3) Canonicalize price tiers + add stable ID
def price_tier_from_price(price):
    try:
        p = float(price)
    except Exception:
        return None
    if p < 50:   return "Budget"
    if p < 100:  return "Mid-range"
    if p < 200:  return "Premium"
    if p < 500:  return "Luxury"
    return "Ultra Luxury"

def canon_tier_name(x):
    s = str(x or "").strip().lower()
    if not s: return ""
    if "ultra" in s: return "Ultra Luxury"
    if "luxury" in s: return "Luxury"
    if "premium" in s: return "Premium"
    if "mid" in s: return "Mid-range"
    if "budget" in s: return "Budget"
    return "Unknown"

def tier_id(name):
    m = {
        "Budget":"budget",
        "Mid-range":"mid",
        "Premium":"premium",
        "Luxury":"luxury",
        "Ultra Luxury":"ultra",
        "Unknown":""
    }
    return m.get(name, "")

# Fill missing/blank tiers from price, then canonicalize
df["price_tier"] = df["price_tier"].where(
    df["price_tier"].notna() & (df["price_tier"].astype(str).str.strip() != "")
)
df.loc[df["price_tier"].isna() | (df["price_tier"].astype(str).str.strip()==""),
       "price_tier"] = df["CHF Price"].apply(price_tier_from_price)

df["Cleaned_Price_Tier"] = df["price_tier"].apply(canon_tier_name)
df["price_tier_id"] = df["Cleaned_Price_Tier"].apply(tier_id)

# 4) Vintage grouping (Power BI-friendly labels) — prefer selected calendar year
try:
    _year_base = int(calendar_year)
except Exception:
    _year_base = datetime.now().year

def classify_vintage_group(vint):
    try:
        s = str(vint).strip().upper()
        if s == "NV": return "Non-Vintage"
        y = int(s)
        cy = _year_base
        if y == cy:       return "Current Vintage"
        if y == cy - 1:   return "En Primeur"
        if cy - y <= 3:   return "Young Wines"
        if cy - y <= 8:   return "Mature Wines"
        if cy - y > 15:   return "Old Wines"
        return "Unknown"
    except Exception:
        return "Unknown"

df["Vintage_Group"] = df["vintage"].apply(classify_vintage_group)

# 5) Order within each tier for triangular/scatter layout
#    Sort by tier (alphabetical with canonical names), then by CHF price desc, then by stock desc, then wine asc
df = df.sort_values(
    ["Cleaned_Price_Tier", "CHF Price", "stock", "wine"],
    ascending=[True, False, False, True],
    na_position="last"
).reset_index(drop=True)

# Tier-local index
df["Tier_Index"] = df.groupby("Cleaned_Price_Tier").cumcount() + 1

# Correct triangular row + centered X within row
# Row r is the smallest r with T_r >= k where T_r = r(r+1)/2
def compute_row(idx):
    # idx is 1-based; use ceil to avoid under-assigning rows
    return int(np.ceil((np.sqrt(8*idx + 1) - 1) / 2.0))

df["Row_Number"] = df["Tier_Index"].apply(compute_row)

# Position within the row: k = idx - T_{r-1}, then center with k - (r+1)/2
def tri_prev(r):  # T_{r-1}
    return r*(r-1)//2

df["k_in_row"] = df["Tier_Index"] - df["Row_Number"].apply(tri_prev)
df["Arrow_Y"] = -df["Row_Number"]
df["Arrow_X_raw"] = df["k_in_row"] - (df["Row_Number"] + 1)/2.0

# Horizontal offsets per tier (keeps tiers separated left→right)
tier_offsets = {
    "Budget": 0,
    "Mid-range": 10,
    "Premium": 20,
    "Luxury": 30,
    "Ultra Luxury": 40
}
df["Tier_X_Offset"] = df["Cleaned_Price_Tier"].map(tier_offsets).fillna(0)
df["Arrow_X"] = df["Arrow_X_raw"] + df["Tier_X_Offset"]

# 6) Select nice columns for Power BI (keep original too)
cols = [
    "id","wine","vintage","Vintage_Group",
    "CHF Price","stock","region_group","full_type",
    "price_tier","Cleaned_Price_Tier","price_tier_id",
    "Tier_Index","Row_Number","Arrow_X","Arrow_Y"
]
present_cols = [c for c in cols if c in df.columns]
out_df = df[present_cols].copy()

# 7) Save to IRON_DATA/OUTPUT_PATH so Power BI can pick it up
out_xlsx = OUTPUT_PATH / "powerbi_wine_arrow_layout.xlsx"
out_csv  = OUTPUT_PATH / "powerbi_wine_arrow_layout.csv"

out_df.to_excel(out_xlsx, index=False)
out_df.to_csv(out_csv, index=False, encoding="utf-8")

print("✅ Power BI layout files saved:")
print("   •", out_xlsx)
print("   •", out_csv)
print("🧭 Rows:", len(out_df), "| Columns:", len(out_df.columns))


✅ Power BI layout files saved:
   • C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\powerbi_wine_arrow_layout.xlsx
   • C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\powerbi_wine_arrow_layout.csv
🧭 Rows: 2389 | Columns: 15


In [32]:
# --- CELL 16: Hydrate weekly cards with history fields (force keys) ---
from pathlib import Path
from datetime import datetime
import json, re

ENABLE_HYDRATION = True
ENABLE_WEEKLY_DEDUP = True

base = Path(OUTPUT_PATH)
yr = int(globals().get("calendar_year", datetime.now().year))
wk = int(globals().get("week_number", datetime.now().isocalendar().week))

# Pick weekly file (year-aware first, legacy fallback)
candidates = [
    base / f"weekly_campaign_schedule_{yr}_week_{wk}.json",
    base / f"weekly_campaign_schedule_week_{wk}.json",
]
weekly = next((p for p in candidates if p.exists()), None)
print(f"📅 Using: {weekly.name if weekly else '—'}  | Exists: {bool(weekly)}")
if not weekly:
    raise SystemExit("No weekly schedule JSON found to hydrate.")

hist_path = base / "history" / "wine_campaign_history.json"
if not hist_path.exists():
    raise SystemExit("No wine_campaign_history.json found. Run the history cell first.")

hist = json.loads(hist_path.read_text(encoding="utf-8"))

# Accept both nested and flat history shapes
if "by_id" in hist:
    last_by_id   = {str(k): (v.get("last_campaign_date","") if isinstance(v,dict) else str(v or "")) for k, v in hist.get("by_id", {}).items()}
    last_by_name = {str(k): (v.get("last_campaign_date","") if isinstance(v,dict) else str(v or "")) for k, v in hist.get("by_name", {}).items()}
    emails_by_id   = {str(k): int(v or 0) for k, v in hist.get("emails_sent_by_id", {}).items()}
    emails_by_name = {str(k): int(v or 0) for k, v in hist.get("emails_sent_by_name", {}).items()}
else:
    # Flat map from Cell 5: { key -> {wine, vintage, last_campaign_date, emails_total} }
    last_by_id, last_by_name, emails_by_id, emails_by_name = {}, {}, {}, {}
    for k, val in hist.items():
        if isinstance(val, dict):
            lcd = val.get("last_campaign_date", "") or ""
            # NOTE: use emails_total (Cell 5) with fallback to emails_sent
            eml = val.get("emails_total", val.get("emails_sent", 0)) or 0
        else:
            lcd, eml = (str(val or ""), 0)
        key = str(k)
        if re.fullmatch(r"\d+", key):
            last_by_id[key] = lcd
            emails_by_id[key] = int(eml)
        else:
            last_by_name[key] = lcd
            emails_by_name[key] = int(eml)

# Optional: supplement from campaign_index.json when available
try:
    ci_path = base / "campaign_index.json"
    if ci_path.exists():
        ci = json.loads(ci_path.read_text(encoding="utf-8"))
        for k, v in (ci.get("emails_sent_by_id", {}) or {}).items():
            emails_by_id[str(k)] = int(v or 0)
        for k, v in (ci.get("emails_sent_by_name", {}) or {}).items():
            emails_by_name[str(k)] = int(v or 0)
except Exception:
    pass


def _name_key(item: dict) -> str:
    w = str(item.get("wine") or item.get("name") or "").strip()
    v = str(item.get("vintage") or "NV").strip()
    return f"{w}::{v}"

data = json.loads(weekly.read_text(encoding="utf-8"))

changed = 0
cards = 0
for day, items in (data.items() if isinstance(data, dict) else []):
    if not isinstance(items, list): 
        continue
    for it in items:
        if not isinstance(it, dict):
            continue
        cards += 1
        wid = str(it.get("id") or "").strip()
        nk  = _name_key(it)

        lcd = (last_by_id.get(wid) or last_by_name.get(nk) or "")
        em  = emails_by_id.get(wid)
        if em is None:
            em = emails_by_name.get(nk, 0)
        em = int(em or 0)

        # detect change before overwriting
        prev_lcd = it.get("last_campaign_date", None)
        prev_em  = it.get("emails_sent", None)
        if prev_lcd != lcd: changed += 1
        if prev_em != em:   changed += 1

        # 🔒 FORCE keys to exist on every card
        it["last_campaign_date"] = lcd
        it["emails_sent"] = em

weekly.write_text(json.dumps(data, indent=2), encoding="utf-8")

# Verify across ALL cards (not just the first)
missing_counts = {"emails_sent": 0, "last_campaign_date": 0}
for day, items in (data.items() if isinstance(data, dict) else []):
    if not isinstance(items, list): 
        continue
    for it in items:
        if "emails_sent" not in it: missing_counts["emails_sent"] += 1
        if "last_campaign_date" not in it: missing_counts["last_campaign_date"] += 1

print(f"🧩 Cards total: {cards}")
print("✅ Hydration:", "updated fields" if changed else "no changes needed")
if any(missing_counts.values()):
    print("❌ Missing keys:", {k:v for k,v in missing_counts.items() if v})
else:
    print("✅ Required keys present on ALL cards: emails_sent, last_campaign_date")


📅 Using: weekly_campaign_schedule_2025_week_38.json  | Exists: True
🧩 Cards total: 35
✅ Hydration: updated fields
✅ Required keys present on ALL cards: emails_sent, last_campaign_date


In [33]:
# --- CELL 17
from pathlib import Path
import json, re
import pandas as pd

base = Path(OUTPUT_PATH)

# ---------------------------
# 1) Weekly file integrity
# ---------------------------
yr = int(globals().get("calendar_year", pd.Timestamp.now().year))
wk = int(globals().get("week_number", pd.Timestamp.now().isocalendar()[1]))

weekly = base / f"weekly_campaign_schedule_{yr}_week_{wk}.json"
if not weekly.exists():
    weekly = base / f"weekly_campaign_schedule_week_{wk}.json"

if not weekly.exists():
    print(f"❌ No weekly schedule JSON found for {yr}-W{wk}. Aborting checks.")
    raise SystemExit(0)

try:
    data = json.loads(weekly.read_text(encoding="utf-8"))
except Exception as e:
    print(f"❌ Failed to read weekly schedule: {e}")
    raise SystemExit(0)

cards = sum(len(v) for v in data.values() if isinstance(v, list))
missing = {"emails_sent": 0, "last_campaign_date": 0}
unknowns = 0
dupe_keys = set()
seen_keys = set()

def _wine_key(c):
    # Prefer id; fallback to wine::vintage
    wid = str(c.get("id") or "").strip()
    if wid:
        return wid
    w = str(c.get("wine") or c.get("name") or "").strip()
    v = str(c.get("vintage") or "NV").strip()
    return f"{w}::{v}"

for day, items in (data.items() if isinstance(data, dict) else []):
    if not isinstance(items, list): 
        continue
    for c in items:
        # required keys
        for k in missing:
            if k not in c:
                missing[k] += 1
        # “Unknown” labels
        if (c.get("wine") in (None,"","Unknown")) or \
           (c.get("region_group") in (None,"","Unknown")) or \
           (c.get("full_type") in (None,"","Unknown")):
            unknowns += 1
        # duplicate detection (per-week)
        key = _wine_key(c)
        if key in seen_keys: 
            dupe_keys.add(key)
        else: 
            seen_keys.add(key)

print(f"📅 Weekly: {weekly.name} | Cards: {cards}")
print("✅ Keys present on all cards" if not any(missing.values()) else f"❌ Missing keys: {missing}")
print(f"ℹ️ Cards with any 'Unknown' label: {unknowns}")
print("✅ No per-week duplicates by wine_key" if not dupe_keys else f"❌ Duplicates by wine_key: {len(dupe_keys)}")

# ---------------------------------------------
# 2) History / campaign_index parity & lookups
# ---------------------------------------------
hist_path = base / "history" / "wine_campaign_history.json"
idx_path  = base / "campaign_index.json"

hist = {}
if hist_path.exists():
    try:
        hist = json.loads(hist_path.read_text(encoding="utf-8"))
    except Exception as e:
        print(f"⚠️ Could not parse history file: {e}")
else:
    print("⚠️ history/wine_campaign_history.json not found (hydration still okay if Cell 16 ran).")

idx = {}
if idx_path.exists():
    try:
        idx = json.loads(idx_path.read_text(encoding="utf-8"))
    except Exception as e:
        print(f"⚠️ Could not parse campaign_index.json: {e}")

def _flat_hist_get_last(k):
    """Tolerate both flat and structured history payloads."""
    if isinstance(hist, dict) and "by_id" in hist:
        v = hist.get("by_id", {}).get(k) or hist.get("by_name", {}).get(k) or {}
        return v.get("last_campaign_date","") if isinstance(v, dict) else (v or "")
    v = hist.get(k, {})
    return v.get("last_campaign_date","") if isinstance(v, dict) else (v or "")

if idx:
    if all(k in idx for k in ("by_id","by_name","meta_by_id")):
        print("✅ campaign_index.json keys present: by_id/by_name/meta_by_id")
    else:
        print("❌ campaign_index.json missing expected keys (by_id/by_name/meta_by_id)")

# spot-check (one mismatch per day max, like a sample)
mismatch = 0
for day, items in (data.items() if isinstance(data, dict) else []):
    if not isinstance(items, list): 
        continue
    for c in items:
        wid = str(c.get("id") or "").strip()
        nk  = f"{str(c.get('wine') or c.get('name') or '').strip()}::{str(c.get('vintage') or 'NV').strip()}"
        lcd_hist = _flat_hist_get_last(wid or nk)
        if (c.get("last_campaign_date") or "") != (lcd_hist or ""):
            mismatch += 1
            break
print("✅ Card last_campaign_date matches history for sampled cards" if mismatch==0 else f"⚠️ {mismatch} day(s) show a mismatch vs history")

# ---------------------------------------------
# 3) Budget coverage sanity (need ≥ 3 per week)
# ---------------------------------------------
def _canon_tier(x):
    s = str(x or "").strip().lower().replace("-", " ")
    if "ultra" in s: return "Ultra Luxury"
    if "luxury" in s: return "Luxury"
    if "premium" in s: return "Premium"
    if "mid" in s: return "Mid-range"
    if "budget" in s: return "Budget"
    return s.title() if s else ""

budget_list = []
for day, items in (data.items() if isinstance(data, dict) else []):
    if not isinstance(items, list): 
        continue
    for c in items:
        tier = _canon_tier(c.get("price_tier"))
        if tier == "Budget":
            budget_list.append((day, c.get("wine") or c.get("name") or c.get("id")))

print(f"💰 Budget offers this week: {len(budget_list)} (target ≥ 3)")
if len(budget_list) < 3:
    print("⚠️ Fewer than 3 Budget offers detected — upstream Cell 12 should have injected extras. Re-run Cell 12 if needed.")
else:
    # quick summary by day
    per_day = {}
    for d, _ in budget_list:
        per_day[d] = per_day.get(d, 0) + 1
    print("   Breakdown by day:", per_day)


📅 Weekly: weekly_campaign_schedule_2025_week_38.json | Cards: 35
✅ Keys present on all cards
ℹ️ Cards with any 'Unknown' label: 10
❌ Duplicates by wine_key: 8
✅ campaign_index.json keys present: by_id/by_name/meta_by_id
✅ Card last_campaign_date matches history for sampled cards
💰 Budget offers this week: 2 (target ≥ 3)
⚠️ Fewer than 3 Budget offers detected — upstream Cell 12 should have injected extras. Re-run Cell 12 if needed.


In [34]:
# --- CELL 18: Weekly de-dup (one card per wine_key; allow Wednesday overflow) ---
import os
from pathlib import Path
import json
import pandas as pd
from datetime import datetime, date

NUM_SLOTS = int(globals().get("NUM_SLOTS", 5))
DAYS_FULL = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

# ---- Config: Wednesday overflow ----
# If WEDNESDAY_MAX_SLOTS is:
#   - A number  -> use that many slots on Wednesday
#   - "0" or "" -> treat as unlimited
# Not set       -> unlimited (i.e., don't clamp Wednesday)
_wed_cap_env = str(os.getenv("WEDNESDAY_MAX_SLOTS", "")).strip()
if _wed_cap_env == "" or _wed_cap_env == "0":
    WEDNESDAY_CAP = None  # unlimited
else:
    try:
        WEDNESDAY_CAP = int(_wed_cap_env)
    except Exception:
        WEDNESDAY_CAP = None  # default unlimited if parse fails

base = Path(OUTPUT_PATH)
yr = int(globals().get("calendar_year", pd.Timestamp.now().year))
wk = int(globals().get("week_number", pd.Timestamp.now().isocalendar()[1]))

weekly = base / f"weekly_campaign_schedule_{yr}_week_{wk}.json"
if not weekly.exists():
    weekly = base / f"weekly_campaign_schedule_week_{wk}.json"

data = json.loads(weekly.read_text(encoding="utf-8"))

def wine_key(card: dict) -> str:
    _id = str(card.get("id") or "").strip()
    if _id:
        return _id
    w = str(card.get("wine") or card.get("name") or "").strip()
    v = str(card.get("vintage") or "NV").strip()
    return f"{w}::{v}"

def parse_date(s):
    if not s: return pd.NaT
    try:
        return pd.to_datetime(s, errors="coerce").date()
    except Exception:
        return pd.NaT

def _to_int_or0(x):
    try:
        v = pd.to_numeric(x, errors="coerce")
        return int(v) if pd.notna(v) else 0
    except Exception:
        return 0

# Flatten with day info for stable ordering
rows = []
for d in DAYS_FULL:
    for c in (data.get(d) or []):
        rows.append({
            "day": d,
            "card": c,
            "wine_key": wine_key(c),
            "locked": bool(c.get("locked", False)),
            "emails_sent": _to_int_or0(c.get("emails_sent")),
            "last_campaign_date": parse_date(c.get("last_campaign_date"))
        })

# Rank: Locked > higher emails_sent > newer last_campaign_date
def rank_tuple(r):
    return (
        1 if r["locked"] else 0,
        r["emails_sent"] or 0,
        r["last_campaign_date"] or date.min
    )

keep_for_key = {}
first_seen_day_order = {d: i for i, d in enumerate(DAYS_FULL)}

for r in rows:
    k = r["wine_key"]
    if k not in keep_for_key:
        keep_for_key[k] = r
    else:
        cur = keep_for_key[k]
        r_rank, c_rank = rank_tuple(r), rank_tuple(cur)
        if r_rank > c_rank:
            keep_for_key[k] = r
        elif r_rank == c_rank:
            # prefer the one that appears earlier in the week
            if first_seen_day_order[r["day"]] < first_seen_day_order[cur["day"]]:
                keep_for_key[k] = r

# Rebuild the calendar: preserve original day order, clamp per-day with Wednesday overflow
deduped = {d: [] for d in DAYS_FULL}
kept = {id(r): True for r in keep_for_key.values()}  # identity set for speed

for d in DAYS_FULL:
    day_items = [r for r in rows if r["day"] == d and kept.get(id(keep_for_key[r["wine_key"]]), False)]
    # keep original order within the day
    deduped[d] = [r["card"] for r in day_items]

    # clamp: all days to NUM_SLOTS, except Wednesday (optional overflow)
    if d != "Wednesday":
        deduped[d] = deduped[d][:NUM_SLOTS]
    else:
        if WEDNESDAY_CAP is not None:  # only clamp if an explicit cap is set
            deduped[d] = deduped[d][:WEDNESDAY_CAP]

# Write back atomically
tmp = weekly.with_suffix(weekly.suffix + ".tmp")
tmp.write_text(json.dumps(deduped, indent=2), encoding="utf-8")
os.replace(tmp, weekly)

# Diagnostics
orig_cards = sum(len(v) for v in data.values() if isinstance(v, list))
new_cards  = sum(len(v) for v in deduped.values() if isinstance(v, list))
dupes_removed = orig_cards - new_cards

print(f"📅 Weekly: {weekly.name}")
print(f"🧽 De-dup complete → removed {dupes_removed} duplicate card(s).")
print("🔢 Cards now per day:", {d: len(deduped[d]) for d in DAYS_FULL})
if WEDNESDAY_CAP is None:
    print("🎉 Wednesday overflow: UNLIMITED")
else:
    print(f"🎉 Wednesday overflow: capped at {WEDNESDAY_CAP}")


📅 Weekly: weekly_campaign_schedule_2025_week_38.json
🧽 De-dup complete → removed 0 duplicate card(s).
🔢 Cards now per day: {'Monday': 5, 'Tuesday': 5, 'Wednesday': 5, 'Thursday': 5, 'Friday': 5, 'Saturday': 5, 'Sunday': 5}
🎉 Wednesday overflow: UNLIMITED


In [36]:
# --- CELL 19: Make `extras` JSON-serializable (deep-safe) ---
import json, math
import numpy as np
import pandas as pd
import datetime as dt
from decimal import Decimal

def _to_py_scalar(o):
    """Convert a single scalar to a JSON-serializable Python type."""
    # pandas / numpy missing
    if o is None or (isinstance(o, float) and math.isnan(o)):
        return None
    try:
        if pd.isna(o):  # handles NaN/NaT/None gracefully
            return None
    except Exception:
        pass

    # pandas time-like
    if isinstance(o, pd.Timestamp):
        return o.isoformat()
    if isinstance(o, pd.Timedelta):
        return o.total_seconds()

    # stdlib time-like
    if isinstance(o, (dt.datetime, dt.date, dt.time)):
        return o.isoformat()

    # numpy scalars
    if isinstance(o, (np.integer,)):
        return int(o)
    if isinstance(o, (np.floating,)):
        return float(o)
    if isinstance(o, (np.bool_,)):
        return bool(o)
    if isinstance(o, (np.datetime64,)):
        ts = pd.to_datetime(o, errors="coerce")
        return ts.isoformat() if pd.notna(ts) else None

    # decimals
    if isinstance(o, Decimal):
        return float(o)

    return o

def _jsonify(o):
    """Deep-convert dict/list/tuple/set/Series/DataFrame into JSON-serializable types."""
    if isinstance(o, dict):
        return {str(k): _jsonify(v) for k, v in o.items()}
    if isinstance(o, (list, tuple, set)):
        return [_jsonify(v) for v in o]
    if isinstance(o, np.ndarray):
        return [_jsonify(v) for v in o.tolist()]
    if isinstance(o, pd.Series):
        return _jsonify(o.to_dict())
    if isinstance(o, pd.DataFrame):
        return [_jsonify(r) for r in o.to_dict(orient="records")]
    # scalar / other
    return _to_py_scalar(o)

# Convert and print
extras_py = _jsonify(extras) if 'extras' in globals() else {}
print("\n+ Extras from stock (optional/tooltips):")
print(json.dumps(extras_py, indent=2, ensure_ascii=False))



+ Extras from stock (optional/tooltips):
{
  "producer": "Marchesi Mazzei Castello di Fonterutoli",
  "chf_price": 975.0,
  "grape_list": "Red Blend",
  "bottle_size_ml": 1800,
  "type_class": "Red",
  "body": 4,
  "sweetness": 3,
  "occasion": "Gifting",
  "avg_score": 95.0
}
